In [2]:
!nvidia-smi

Fri Jul 17 18:54:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Dwonload files config.py, utils.py, datasets.py, pretrain_DAMSM.py
!git clone https://github.com/ammarnasr/CUB-Attn-GAN.git
#Move Models text and image encoder to their /content/
!mv /content/CUB-Attn-GAN/theNotebook/only_captions.csv /content/
#Remove the Repo
!rm -r /content/CUB-Attn-GAN/

In [ ]:
!pip install transformers


import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer , BertModel
from torch.utils.data import TensorDataset, random_split



if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


df = pd.read_csv("/content/only_captions.csv", header=None)
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
sentences = df[1].values
sentences = np.delete(sentences, 0)


print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_ids = []
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(sent, add_special_tokens = True, max_length = 18, pad_to_max_length = True, return_attention_mask = True,   
                        return_tensors = 'pt', truncation=True)
    input_ids.append(encoded_dict['input_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
segments_ids = []
for i in range(input_ids.size()[0]):
  segments_id = [1] * input_ids.size()[1]
  segments_ids.append(segments_id)
segments_ids = torch.tensor(segments_ids)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])


In [6]:
import os
os.chdir('/content')
!rm -r sample_data

#clone repo AttnGAN
!git clone https://github.com/taoxugit/AttnGAN.git

#Changing Working dirctory to data
os.chdir('/content/AttnGAN/data/')
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ' -O birds.zip
!unzip -q birds.zip
!rm birds.zip
!rm -r __MACOSX/

#Changing Working dirctory to birds
os.chdir('/content/AttnGAN/data/birds/')
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45" -O CUB_200_2011.tgz && rm -rf /tmp/cookies.txt
!tar zxf  CUB_200_2011.tgz
!rm CUB_200_2011.tgz

#Changing Working dirctory to code
os.chdir('/content/AttnGAN/code/')
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Wr3lQajG7m6Bi3rYFTJb6mwE_d8su111' -O Pillow.rar
!unrar x  Pillow.rar
!rm Pillow.rar

Cloning into 'AttnGAN'...
remote: Enumerating objects: 291, done.
remote: Total 291 (delta 0), reused 0 (delta 0), pack-reused 291
Receiving objects: 100% (291/291), 36.76 MiB | 35.95 MiB/s, done.
Resolving deltas: 100% (167/167), done.
--2020-07-17 19:05:46--  https://docs.google.com/uc?export=download&id=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ
Resolving docs.google.com (docs.google.com)... 74.125.195.102, 74.125.195.101, 74.125.195.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1s1iap1hns7v4khk9s0urlagl42ds0rn/1595012775000/09657060183789739732/*/1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ?e=download [following]
--2020-07-17 19:06:15--  https://doc-0o-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1s1iap1hns7v4khk9s0urlagl42ds0rn/1595012775000/09657060183789739732/*

In [ ]:
import os
os.chdir('/content/AttnGAN/code/')

import os.path as osp
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import numpy as np
import numpy.random as random
import pandas as pd
import pickle
from easydict import EasyDict as edict

from PIL import Image, ImageDraw, ImageFont
from copy import deepcopy
import skimage.transform

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from torchvision import models
import torchvision.transforms as transforms

import torch.utils.model_zoo as model_zoo
import torch.utils.data as data

from nltk.tokenize import RegexpTokenizer
from collections import defaultdict


__C = edict()
cfg = __C
__C.DATASET_NAME = 'birds'
__C.CONFIG_NAME = 'DAMSM'
__C.DATA_DIR = '../data/birds'
__C.GPU_ID = 0
__C.CUDA = True
__C.WORKERS = 1
__C.B_VALIDATION = False

__C.TREE = edict()
__C.TREE.BRANCH_NUM = 1
__C.TREE.BASE_SIZE = 299

# Training options
__C.TRAIN = edict()
__C.TRAIN.BATCH_SIZE = 48
__C.TRAIN.MAX_EPOCH = 600
__C.TRAIN.SNAPSHOT_INTERVAL = 50
__C.TRAIN.DISCRIMINATOR_LR = 0.0002
__C.TRAIN.GENERATOR_LR = 0.0002
__C.TRAIN.ENCODER_LR = 0.002
__C.TRAIN.RNN_GRAD_CLIP = 0.25
__C.TRAIN.FLAG = True
__C.TRAIN.NET_E = ''
__C.TRAIN.NET_G = ''
__C.TRAIN.B_NET_D = True
__C.TRAIN.SMOOTH = edict()
__C.TRAIN.SMOOTH.GAMMA1 = 4.0
__C.TRAIN.SMOOTH.GAMMA3 = 10.0
__C.TRAIN.SMOOTH.GAMMA2 = 5.0
__C.TRAIN.SMOOTH.LAMBDA = 1.0

# Modal options
__C.GAN = edict()
__C.GAN.DF_DIM = 64
__C.GAN.GF_DIM = 128
__C.GAN.Z_DIM = 100
__C.GAN.CONDITION_DIM = 100
__C.GAN.R_NUM = 2
__C.GAN.B_ATTENTION = True
__C.GAN.B_DCGAN = False

__C.TEXT = edict()
__C.TEXT.CAPTIONS_PER_IMAGE = 10
__C.TEXT.EMBEDDING_DIM = 256
__C.TEXT.WORDS_NUM = 18



def get_imgs(img_path, imsize, bbox=None, transform=None, normalize=None):
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        r = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - r)
        y2 = np.minimum(height, center_y + r)
        x1 = np.maximum(0, center_x - r)
        x2 = np.minimum(width, center_x + r)
        img = img.crop([x1, y1, x2, y2])

    if transform is not None:
        img = transform(img)

    ret = []
    if cfg.GAN.B_DCGAN:
        ret = [normalize(img)]
    else:
        for i in range(cfg.TREE.BRANCH_NUM):
            # print(imsize[i])
            if i < (cfg.TREE.BRANCH_NUM - 1):
                re_img = transforms.Scale(imsize[i])(img)
            else:
                re_img = img
            ret.append(normalize(re_img))

    return ret

def prepare_data(data):
    imgs, captions, captions_lens, class_ids, keys, input_ids, segments_ids = data

    # sort data by the length in a decreasing order !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!MARKER!!!!!!!!!!!!!!!!!!!!!!!!
    sorted_cap_lens, sorted_cap_indices = torch.sort(captions_lens, 0, True)

    real_imgs = []
    for i in range(len(imgs)):
        imgs[i] = imgs[i][sorted_cap_indices]
        if cfg.CUDA:
            real_imgs.append(Variable(imgs[i]).cuda())
        else:
            real_imgs.append(Variable(imgs[i]))

    captions = captions[sorted_cap_indices].squeeze()
    class_ids = class_ids[sorted_cap_indices].numpy()
    # sent_indices = sent_indices[sorted_cap_indices]
    keys = [keys[i] for i in sorted_cap_indices.numpy()]
    # print('keys', type(keys), keys[-1])  # list
    if cfg.CUDA:
        captions = Variable(captions).cuda()
        sorted_cap_lens = Variable(sorted_cap_lens).cuda()
    else:
        captions = Variable(captions)
        sorted_cap_lens = Variable(sorted_cap_lens)

    input_ids = input_ids.cuda()
    segments_ids = segments_ids.cuda()

    return [real_imgs, captions, sorted_cap_lens,
            class_ids, keys, input_ids, segments_ids]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def build_super_images(real_imgs, captions, ixtoword,
                        attn_maps, att_sze, lr_imgs=None,
                        batch_size=cfg.TRAIN.BATCH_SIZE,
                        max_word_num=cfg.TEXT.WORDS_NUM):
    
    
    COLOR_DIC = {0:[128,64,128],  1:[244, 35,232],
                2:[70, 70, 70],  3:[102,102,156],
                4:[190,153,153], 5:[153,153,153],
                6:[250,170, 30], 7:[220, 220, 0],
                8:[107,142, 35], 9:[152,251,152],
                10:[70,130,180], 11:[220,20, 60],
                12:[255, 0, 0],  13:[0, 0, 142],
                14:[119,11, 32], 15:[0, 60,100],
                16:[0, 80, 100], 17:[0, 0, 230],
                18:[0,  0, 70],  19:[0, 0,  0]}
    FONT_MAX = 50

    
    build_super_images_start_time = time.time()
    nvis = 8
    real_imgs = real_imgs[:nvis]
    if lr_imgs is not None:
        lr_imgs = lr_imgs[:nvis]
    if att_sze == 17:
        vis_size = att_sze * 16
    else:
        vis_size = real_imgs.size(2)

    text_convas = \
        np.ones([batch_size * FONT_MAX,
                 (max_word_num + 2) * (vis_size + 2), 3],
                dtype=np.uint8)


    # print("keyword |||||||||||||||||||||||||||||||")
    # print("max_word_num : " , max_word_num)
    # print("keyword |||||||||||||||||||||||||||||||")
    for i in range(max_word_num):
        istart = (i + 2) * (vis_size + 2)
        iend = (i + 3) * (vis_size + 2)
        text_convas[:, istart:iend, :] = COLOR_DIC[i]


    real_imgs = \
        nn.Upsample(size=(vis_size, vis_size), mode='bilinear')(real_imgs)
    # [-1, 1] --> [0, 1]
    real_imgs.add_(1).div_(2).mul_(255)
    real_imgs = real_imgs.data.numpy()
    # b x c x h x w --> b x h x w x c
    real_imgs = np.transpose(real_imgs, (0, 2, 3, 1))
    pad_sze = real_imgs.shape
    middle_pad = np.zeros([pad_sze[2], 2, 3])
    post_pad = np.zeros([pad_sze[1], pad_sze[2], 3])
    if lr_imgs is not None:
        lr_imgs = \
            nn.Upsample(size=(vis_size, vis_size), mode='bilinear')(lr_imgs)
        # [-1, 1] --> [0, 1]
        lr_imgs.add_(1).div_(2).mul_(255)
        lr_imgs = lr_imgs.data.numpy()
        # b x c x h x w --> b x h x w x c
        lr_imgs = np.transpose(lr_imgs, (0, 2, 3, 1))

    # batch x seq_len x 17 x 17 --> batch x 1 x 17 x 17
    seq_len = max_word_num
    img_set = []
    num = nvis  # len(attn_maps)

    text_map, sentences = \
        drawCaption(text_convas, captions, ixtoword, vis_size)
    text_map = np.asarray(text_map).astype(np.uint8)

    bUpdate = 1
    for i in range(num):
        #print ("loop " , i ," of " , num == 8)
        attn = attn_maps[i].cpu().view(1, -1, att_sze, att_sze)
        # --> 1 x 1 x 17 x 17
        attn_max = attn.max(dim=1, keepdim=True)
        attn = torch.cat([attn_max[0], attn], 1)
        #
        attn = attn.view(-1, 1, att_sze, att_sze)
        attn = attn.repeat(1, 3, 1, 1).data.numpy()
        # n x c x h x w --> n x h x w x c
        attn = np.transpose(attn, (0, 2, 3, 1))
        num_attn = attn.shape[0]
        #
        img = real_imgs[i]
        if lr_imgs is None:
            lrI = img
        else:
            lrI = lr_imgs[i]
        
        row = [lrI, middle_pad]
        #print("rowwwwwwwwwwwwwwwww : ", row)
        row_merge = [img, middle_pad]
        row_beforeNorm = []
        minVglobal, maxVglobal = 1, 0
        for j in range(num_attn):
            #print ("looop " , j , " of " , seq_len+1)
            one_map = attn[j]
            #print("First one map : " , one_map.shape)
            #print("attn.shape : " , attn.shape)

            
            # print("if (vis_size // att_sze) > 1: " ,  (vis_size // att_sze) > 1)
            # print("vis_size : " , vis_size)
            # print("att_sze : " , att_sze)
            # print("vis_size//att_sze : " , vis_size//att_sze)
            
            if (vis_size // att_sze) > 1:
                one_map = \
                    skimage.transform.pyramid_expand(one_map, sigma=20,
                                                     upscale=vis_size // att_sze)
            #    print("one_map in if : " , one_map.shape)

            
            row_beforeNorm.append(one_map)
            #print("row_beforeNorm.append(one_map)" ,len(row_beforeNorm))
            minV = one_map.min()
            maxV = one_map.max()
            if minVglobal > minV:
                minVglobal = minV
            if maxVglobal < maxV:
                maxVglobal = maxV
            #print("seq_len : " , seq_len)
        for j in range(seq_len + 1):
            #print ("loooop " , j , " of " , seq_len+1)
            
            if j < num_attn:
                one_map = row_beforeNorm[j]
                one_map = (one_map - minVglobal) / (maxVglobal - minVglobal)
                one_map *= 255
                #
                # print ("PIL_im = " , Image.fromarray(np.uint8(img)))
                # print ("PIL_att = " , Image.fromarray(np.uint8(one_map[:,:,:3])))
                # print ("img.size( :" , img.shape)
                # print ("one_map.size( :" , one_map.shape)
                PIL_im = Image.fromarray(np.uint8(img))
                PIL_att = Image.fromarray(np.uint8(one_map[:,:,:3]))
                merged = \
                    Image.new('RGBA', (vis_size, vis_size), (0, 0, 0, 0))
                #print ("merged : " , merged.size)
                mask = Image.new('L', (vis_size, vis_size), (210))
                #print (" mask  : " , mask.size)
                merged.paste(PIL_im, (0, 0))
                #print (" merged.paste(PIL_im)  : " , merged.size )
                ############################################################
                merged.paste(PIL_att, (0, 0), mask)
                #print (" merged.paste(PIL_att)  : " ,  merged.size)#########################
                merged = np.array(merged)[:, :, :3]
                #print ("  np.array(merged)[:::3] : " , merged.size )#########################
                ############################################################
            else:
                #print (" IN THE ELSE post_pad : " , post_pad.shape)
                one_map = post_pad
                #print (" one_map  : " , one_map.shape )
                merged = post_pad
                #print ("  OUTTING THE ELSE : " , merged.shape )
            
            #print ("  row : " , len(row))
            row.append(one_map[:,:,:3])
            #print ("  row.appedn(one_map) : " , len(row))
            row.append(middle_pad)
            #print ("  row.append(middle_pad) : " , len(row))
            #
            #print ("  row_merge : " , len(row_merge))
            row_merge.append(merged)
            #print ("  row_merge.append(mereged) : " , len(row_merge) )
            row_merge.append(middle_pad)
            #print ("  row_merge.append(middle_pad) : " , len(row_merge) )
        ####################################################################
        # print("row.shape : ", len(row))
        # for i in range(len(row)):
        #   print('arr', i,   
        #         " => dim0:", len(row[i]),
        #         " || dim1:", len(row[i][0]),
        #         " || dim2:", len(row[i][0][0]))
        # #print(row)
        # print("row[0].shape : ", len(row[0]))
        # #print(row[0])
        # print("row[0][0].shape : ", len(row[0][0]))
        # #print(row[0][0])
        # print("row[0][0][0].shape : ", len(row[0][0][0]))
        # #print(row[0][0][0])

        # print("row[1].shape : ", len(row[1]))
        # #print(row[1])
        # print("row[1][0].shape : ", len(row[1][0]))
        # #print(row[1][0])
        # print("row[1][0][0].shape : ", len(row[1][0][0]))
        # #print(row[1][0][0])

        # print("row[2].shape : ", len(row[2]))
        # #print(row[2])
        # print("row[2][0].shape : ", len(row[2][0]))
        # #print(row[2][0])
        # print("row[2][0][0].shape : ", len(row[2][0][0]))
        # #print(row[2][0][0])

        # print("row[3].shape : ", len(row[3]))
        # #print(row[2])
        # print("row[3][0].shape : ", len(row[3][0]))
        # #print(row[2][0])
        # print("row[3][0][0].shape : ", len(row[3][0][0]))
        # #print(row[2][0][0])

        # print("row[4].shape : ", len(row[4]))
        # #print(row[2])
        # print("row[4][0].shape : ", len(row[4][0]))
        # #print(row[2][0])
        # print("row[4][0][0].shape : ", len(row[4][0][0]))
        #print(row[2][0][0])

        

        
        
        row = np.concatenate(row, 1)
        #print (" row.conatent(1)  : " ,  len(row))########################################
        row_merge = np.concatenate(row_merge, 1)
        #print ("   : " , )############################
        ####################################################################
        txt = text_map[i * FONT_MAX: (i + 1) * FONT_MAX]
        if txt.shape[1] != row.shape[1]:
            print('txt', txt.shape, 'row', row.shape)
            bUpdate = 0
            break
        #####################################################################
        row = np.concatenate([txt, row, row_merge], 0)#######################
        img_set.append(row)##################################################
        #####################################################################
    
    # print("keyword |||||||||||||||||||||||||||||||")
    # print("bUpdate : " , bUpdate)
    # print("keyword |||||||||||||||||||||||||||||||")
    if bUpdate:
        img_set = np.concatenate(img_set, 0)
        img_set = img_set.astype(np.uint8)
        print("keyTime |||||||||||||||||||||||||||||||")
        print("build_super_images_time : " , time.time() - build_super_images_start_time)
        print("KeyTime |||||||||||||||||||||||||||||||")
        return img_set, sentences
    else:
        print("keyTime |||||||||||||||||||||||||||||||")
        print("build_super_images_start_time : " , time.time() - build_super_images_start_time)
        print("KeyTime |||||||||||||||||||||||||||||||")
        return None

def conv1x1(in_planes, out_planes, bias=False):
    "1x1 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                     padding=0, bias=bias)


class TextDataset(data.Dataset):
    def __init__(self, data_dir, split='train',
                    base_size=64,
                    transform=None, target_transform=None, input_ids=None, segments_ids=None, sentences=None):
        self.transform = transform
        self.norm = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.target_transform = target_transform
        self.embeddings_num = cfg.TEXT.CAPTIONS_PER_IMAGE

        self.imsize = []# [299]
        for i in range(cfg.TREE.BRANCH_NUM):
            self.imsize.append(base_size)
            base_size = base_size * 2
        print("self.imsize", self.imsize)

        self.data = []
        self.data_dir = data_dir
        if data_dir.find('birds') != -1:
            self.bbox = self.load_bbox() # 11788 long dictionry with key as image name and value is 4 ints list bounding box
        else:
            self.bbox = None
        split_dir = os.path.join(data_dir, split)

        self.filenames, self.captions, self.ixtoword, self.wordtoix, self.n_words = self.load_text_data(data_dir, split)
        #filenames: List of 8855 text items of image names
        #captions: List of 8855 varible lengths captions -in range 9-18 -
        #ixtoword: dictionry  of 5450 index [key] to word [value] pairs
        #wordtoix: dictionry  of 5450 word [key] to index [value] pairs
        #n_words: 5450

        self.class_id = self.load_class_id(split_dir, len(self.filenames)) #200 classes, len:8855

        self.number_example = len(self.filenames) #8855

        self.input_ids = input_ids
        self.segments_ids = segments_ids
        self.sentences = sentences

    def load_bbox(self):
        data_dir = self.data_dir
        bbox_path = os.path.join(data_dir, 'CUB_200_2011/bounding_boxes.txt')
        df_bounding_boxes = pd.read_csv(bbox_path,
                                        delim_whitespace=True,
                                        header=None).astype(int)
        #
        filepath = os.path.join(data_dir, 'CUB_200_2011/images.txt')
        df_filenames = pd.read_csv(filepath, delim_whitespace=True, header=None)
        filenames = df_filenames[1].tolist()
        print('Total filenames: ', len(filenames), filenames[0])
        #
        filename_bbox = {img_file[:-4]: [] for img_file in filenames}
        numImgs = len(filenames)
        for i in range(0, numImgs):
            # bbox = [x-left, y-top, width, height]
            bbox = df_bounding_boxes.iloc[i][1:].tolist()

            key = filenames[i][:-4]
            filename_bbox[key] = bbox
        #
        return filename_bbox

    def load_captions(self, data_dir, filenames):
        all_captions = []
        for i in range(len(filenames)):
            cap_path = '%s/text/%s.txt' % (data_dir, filenames[i])
            with open(cap_path, "r") as f:
                captions = f.read().decode('utf8').split('\n')
                cnt = 0
                for cap in captions:
                    if len(cap) == 0:
                        continue
                    cap = cap.replace("\ufffd\ufffd", " ")
                    # picks out sequences of alphanumeric characters as tokens
                    # and drops everything else
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(cap.lower())
                    # print('tokens', tokens)
                    if len(tokens) == 0:
                        print('cap', cap)
                        continue

                    tokens_new = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0:
                            tokens_new.append(t)
                    all_captions.append(tokens_new)
                    cnt += 1
                    if cnt == self.embeddings_num:
                        break
                if cnt < self.embeddings_num:
                    print('ERROR: the captions for %s less than %d'% (filenames[i], cnt))
        return all_captions

    def build_dictionary(self, train_captions, test_captions):
        word_counts = defaultdict(float)
        captions = train_captions + test_captions
        for sent in captions:
            for word in sent:
                word_counts[word] += 1

        vocab = [w for w in word_counts if word_counts[w] >= 0]

        ixtoword = {}
        ixtoword[0] = '<end>'
        wordtoix = {}
        wordtoix['<end>'] = 0
        ix = 1
        for w in vocab:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1

        train_captions_new = []
        for t in train_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            train_captions_new.append(rev)

        test_captions_new = []
        for t in test_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            test_captions_new.append(rev)

        return [train_captions_new, test_captions_new,
                ixtoword, wordtoix, len(ixtoword)]

    def load_text_data(self, data_dir, split):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = self.load_filenames(data_dir, 'train')
        test_names = self.load_filenames(data_dir, 'test')
        if not os.path.isfile(filepath):
            train_captions = self.load_captions(data_dir, train_names)
            test_captions = self.load_captions(data_dir, test_names)

            train_captions, test_captions, ixtoword, wordtoix, n_words = \
                self.build_dictionary(train_captions, test_captions)
            with open(filepath, 'wb') as f:
                pickle.dump([train_captions, test_captions,
                                ixtoword, wordtoix], f, protocol=2)
                print('Save to: ', filepath)
        else:
            with open(filepath, 'rb') as f:
                x = pickle.load(f)
                train_captions, test_captions = x[0], x[1]
                ixtoword, wordtoix = x[2], x[3]
                del x
                n_words = len(ixtoword)
                print('Load from: ', filepath)
        if split == 'train':
            # a list of list: each list contains
            # the indices of words in a sentence
            captions = train_captions
            filenames = train_names
        else:  # split=='test'
            captions = test_captions
            filenames = test_names
        return filenames, captions, ixtoword, wordtoix, n_words

    def load_class_id(self, data_dir, total_num):
        if os.path.isfile(data_dir + '/class_info.pickle'):
            with open(data_dir + '/class_info.pickle', 'rb') as f:
                class_id = pickle.load(f , encoding = 'latin1')
        else:
            class_id = np.arange(total_num)
        return class_id

    def load_filenames(self, data_dir, split):
        filepath = '%s/%s/filenames.pickle' % (data_dir, split)
        if os.path.isfile(filepath):
            with open(filepath, 'rb') as f:
                filenames = pickle.load(f)
            print('Load filenames from: %s (%d)' % (filepath, len(filenames)))
        else:
            filenames = []
        return filenames

    def get_caption(self, sent_ix):
        # a list of indices for a sentence
        sent_caption = np.asarray(self.captions[sent_ix]).astype('int64')
        if (sent_caption == 0).sum() > 0:
            print('ERROR: do not need END (0) token', sent_caption)
        num_words = len(sent_caption)
        # pad with 0s (i.e., '<end>')
        x = np.zeros((cfg.TEXT.WORDS_NUM, 1), dtype='int64')
        x_len = num_words
        if num_words <= cfg.TEXT.WORDS_NUM:
            x[:num_words, 0] = sent_caption
        else:
            ix = list(np.arange(num_words))  # 1, 2, 3,..., maxNum
            np.random.shuffle(ix)
            ix = ix[:cfg.TEXT.WORDS_NUM]
            ix = np.sort(ix)
            x[:, 0] = sent_caption[ix]
            x_len = cfg.TEXT.WORDS_NUM
        return x, x_len

    def __getitem__(self, index):
        #
        key = self.filenames[index]
        cls_id = self.class_id[index]
        #
        if self.bbox is not None:
            bbox = self.bbox[key]
            data_dir = '%s/CUB_200_2011' % self.data_dir
        else:
            bbox = None
            data_dir = self.data_dir
        #
        img_name = '%s/images/%s.jpg' % (data_dir, key)
        imgs = get_imgs(img_name, self.imsize,
                        bbox, self.transform, normalize=self.norm)
        # random select a sentence
        sent_ix = random.randint(0, self.embeddings_num)
        new_sent_ix = index * self.embeddings_num + sent_ix
        caps, cap_len = self.get_caption(new_sent_ix)

        caps_dec = []
        for word in caps:
            caps_dec.append(self.ixtoword[int(word)])

        return imgs, caps, cap_len, cls_id, key, self.input_ids[index], self.segments_ids[index]


    def __len__(self):
        return len(self.filenames)


class CNN_ENCODER(nn.Module):
    def __init__(self, nef):
        super(CNN_ENCODER, self).__init__()
        if cfg.TRAIN.FLAG:
            self.nef = nef
        else:
            self.nef = 256  # define a uniform ranker
        
        print('CNN_ENCODER')
        model = models.inception_v3()
        print('loaded_inception')
        url = 'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'
        model.load_state_dict(model_zoo.load_url(url))
        for param in model.parameters():
            param.requires_grad = False
        print('Load pretrained model from ', url)
        # print(model)

        self.define_module(model)
        self.init_trainable_weights()

    def define_module(self, model):
        self.Conv2d_1a_3x3 = model.Conv2d_1a_3x3
        self.Conv2d_2a_3x3 = model.Conv2d_2a_3x3
        self.Conv2d_2b_3x3 = model.Conv2d_2b_3x3
        self.Conv2d_3b_1x1 = model.Conv2d_3b_1x1
        self.Conv2d_4a_3x3 = model.Conv2d_4a_3x3
        self.Mixed_5b = model.Mixed_5b
        self.Mixed_5c = model.Mixed_5c
        self.Mixed_5d = model.Mixed_5d
        self.Mixed_6a = model.Mixed_6a
        self.Mixed_6b = model.Mixed_6b
        self.Mixed_6c = model.Mixed_6c
        self.Mixed_6d = model.Mixed_6d
        self.Mixed_6e = model.Mixed_6e
        self.Mixed_7a = model.Mixed_7a
        self.Mixed_7b = model.Mixed_7b
        self.Mixed_7c = model.Mixed_7c

        self.emb_features = conv1x1(768, self.nef)
        self.emb_cnn_code = nn.Linear(2048, self.nef)

    def init_trainable_weights(self):
        initrange = 0.1
        self.emb_features.weight.data.uniform_(-initrange, initrange)
        self.emb_cnn_code.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        features = None
        # --> fixed-size input: batch x 3 x 299 x 299
        x = nn.Upsample(size=(299, 299), mode='bilinear')(x)
        # 299 x 299 x 3
        x = self.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.Conv2d_4a_3x3(x)
        # 71 x 71 x 192

        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.Mixed_5d(x)
        # 35 x 35 x 288

        x = self.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.Mixed_6e(x)
        # 17 x 17 x 768

        # image region features
        features = x
        # 17 x 17 x 768

        x = self.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=8)
        # 1 x 1 x 2048
        # x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048

        # global image features
        cnn_code = self.emb_cnn_code(x)
        # 512
        if features is not None:
            features = self.emb_features(features)
        return features, cnn_code

class BERT_ENCODER(nn.Module):
    def __init__(self):
        super(BERT_ENCODER, self).__init__()

        
        print('Bert_ENCODER')
        
        model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
        for param in model.parameters():
            param.requires_grad = False
        print('Load pretrained model from  BertModel')

        self.define_module(model)
        self.init_trainable_weights()
        self.bert_model = model

    def define_module(self, model):
        self.word_bert_code = nn.Linear(768, 256)
        self.sent_bert_code = nn.Linear(768, 256)

    def init_trainable_weights(self):
        initrange = 0.1
        self.word_bert_code.weight.data.uniform_(-initrange, initrange)
        self.sent_bert_code.weight.data.uniform_(-initrange, initrange)

    def forward(self,  b_input_ids, b_segments_ids):
        
        outputs = self.bert_model(b_input_ids, b_segments_ids)
        hidden_states = outputs[2]

        word_embedding = torch.stack(hidden_states, dim=0)


        word_embedding= self.word_bert_code(word_embedding)


        word_embedding = word_embedding.permute(1,3,0,2)

        word_embedding = word_embedding.sum(dim=2)



        token_vecs = hidden_states[-2]

        # Calculate the average of all 18 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=1)
        sentence_embedding= self.sent_bert_code(sentence_embedding)


        return  word_embedding, sentence_embedding



def drawCaption(convas, captions, ixtoword, vis_size, off1=2, off2=2):
    
    FONT_MAX = 50

    num = captions.size(0)
    img_txt = Image.fromarray(convas)
    # get a font
    # fnt = None  # ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 50)
    print ("CURRENT WORKING DIRCTORY : " , os.getcwd())
    fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 50)
    # get a drawing context
    d = ImageDraw.Draw(img_txt)
    sentence_list = []
    for i in range(num):
        cap = captions[i].data.cpu().numpy()
        sentence = []
        for j in range(len(cap)):
            if cap[j] == 0:
                break
            word = ixtoword[cap[j]].encode('ascii', 'ignore').decode('ascii')
            d.text(((j + off1) * (vis_size + off2), i * FONT_MAX), '%d:%s' % (j, word[:6]),
                   font=fnt, fill=(255, 255, 255, 255))
            sentence.append(word)
        sentence_list.append(sentence)
    return img_txt, sentence_list

def cosine_similarity(x1, x2, dim=1, eps=1e-8):
    """Returns cosine similarity between x1 and x2, computed along dim.
    """
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return (w12 / (w1 * w2).clamp(min=eps)).squeeze()

def sent_loss(cnn_code, rnn_code, labels, class_ids, batch_size, eps=1e-8):
    # ### Mask mis-match samples  ###
    # that come from the same class as the real sample ###
    masks = []
    if class_ids is not None:
        for i in range(batch_size):
            mask = (class_ids == class_ids[i]).astype(np.uint8)
            mask[i] = 0
            masks.append(mask.reshape((1, -1)))
        masks = np.concatenate(masks, 0)
        # masks: batch_size x batch_size
        masks = torch.BoolTensor(masks)
        if cfg.CUDA:
            masks = masks.cuda()

    # --> seq_len x batch_size x nef
    if cnn_code.dim() == 2:
        cnn_code = cnn_code.unsqueeze(0)
        rnn_code = rnn_code.unsqueeze(0)

    # cnn_code_norm / rnn_code_norm: seq_len x batch_size x 1
    cnn_code_norm = torch.norm(cnn_code, 2, dim=2, keepdim=True)
    rnn_code_norm = torch.norm(rnn_code, 2, dim=2, keepdim=True)
    # scores* / norm*: seq_len x batch_size x batch_size
    scores0 = torch.bmm(cnn_code, rnn_code.transpose(1, 2))
    norm0 = torch.bmm(cnn_code_norm, rnn_code_norm.transpose(1, 2))
    scores0 = scores0 / norm0.clamp(min=eps) * cfg.TRAIN.SMOOTH.GAMMA3

    # --> batch_size x batch_size
    scores0 = scores0.squeeze()
    if class_ids is not None:
        scores0.data.masked_fill_(masks, -float('inf'))
    scores1 = scores0.transpose(0, 1)
    if labels is not None:
        loss0 = nn.CrossEntropyLoss()(scores0, labels)
        loss1 = nn.CrossEntropyLoss()(scores1, labels)
    else:
        loss0, loss1 = None, None
    return loss0, loss1


def words_loss(img_features, words_emb, labels, cap_lens, class_ids, batch_size):
    """
        words_emb(query): batch x nef x seq_len
        img_features(context): batch x nef x 17 x 17
    """
    masks = []
    att_maps = []
    similarities = []
    cap_lens = cap_lens.data.tolist()

    for i in range(batch_size):
    
        if class_ids is not None:
            mask = (class_ids == class_ids[i]).astype(np.uint8)
            mask[i] = 0
            masks.append(mask.reshape((1, -1)))
        # Get the i-th text description
        words_num = cap_lens[i]
        # -> 1 x nef x words_num
        word = words_emb[i, :, :words_num].unsqueeze(0).contiguous()
        # -> batch_size x nef x words_num
        word = word.repeat(batch_size, 1, 1)
        # batch x nef x 17*17
        context = img_features
        """
            word(query): batch x nef x words_num
            context: batch x nef x 17 x 17
            weiContext: batch x nef x words_num
            attn: batch x words_num x 17 x 17
        """
        weiContext, attn = func_attention(word, context, cfg.TRAIN.SMOOTH.GAMMA1)
        att_maps.append(attn[i].unsqueeze(0).contiguous())
        # --> batch_size x words_num x nef
        word = word.transpose(1, 2).contiguous()
        weiContext = weiContext.transpose(1, 2).contiguous()
        # --> batch_size*words_num x nef
        word = word.view(batch_size * words_num, -1)
        weiContext = weiContext.view(batch_size * words_num, -1)
        #
        # -->batch_size*words_num
        row_sim = cosine_similarity(word, weiContext)
        # --> batch_size x words_num
        row_sim = row_sim.view(batch_size, words_num)

        # Eq. (10)
        row_sim.mul_(cfg.TRAIN.SMOOTH.GAMMA2).exp_()
        row_sim = row_sim.sum(dim=1, keepdim=True)
        row_sim = torch.log(row_sim)

        # --> 1 x batch_size
        # similarities(i, j): the similarity between the i-th image and the j-th text description
        similarities.append(row_sim)


    # batch_size x batch_size
    similarities = torch.cat(similarities, 1)
    if class_ids is not None:
        masks = np.concatenate(masks, 0)
        # masks: batch_size x batch_size
        masks = torch.BoolTensor(masks)
        if cfg.CUDA:
            masks = masks.cuda()

    similarities = similarities * cfg.TRAIN.SMOOTH.GAMMA3
    if class_ids is not None:
        similarities.data.masked_fill_(masks, -float('inf'))
    similarities1 = similarities.transpose(0, 1)
    if labels is not None:
        loss0 = nn.CrossEntropyLoss()(similarities, labels)
        loss1 = nn.CrossEntropyLoss()(similarities1, labels)
    else:
        loss0, loss1 = None, None
    return loss0, loss1, att_maps

def func_attention(query, context, gamma1):
    """
    query: batch x ndf x queryL
    context: batch x ndf x ih x iw (sourceL=ihxiw)
    mask: batch_size x sourceL
    """
    batch_size, queryL = query.size(0), query.size(2)
    ih, iw = context.size(2), context.size(3)
    sourceL = ih * iw

    # --> batch x sourceL x ndf
    context = context.view(batch_size, -1, sourceL)
    contextT = torch.transpose(context, 1, 2).contiguous()

    # Get attention
    # (batch x sourceL x ndf)(batch x ndf x queryL)
    # -->batch x sourceL x queryL
    attn = torch.bmm(contextT, query) # Eq. (7) in AttnGAN paper
    # --> batch*sourceL x queryL
    attn = attn.view(batch_size*sourceL, queryL)
    attn = nn.Softmax()(attn)  # Eq. (8)

    # --> batch x sourceL x queryL
    attn = attn.view(batch_size, sourceL, queryL)
    # --> batch*queryL x sourceL
    attn = torch.transpose(attn, 1, 2).contiguous()
    attn = attn.view(batch_size*queryL, sourceL)
    #  Eq. (9)
    attn = attn * gamma1
    attn = nn.Softmax()(attn)
    attn = attn.view(batch_size, queryL, sourceL)
    # --> batch x sourceL x queryL
    attnT = torch.transpose(attn, 1, 2).contiguous()

    # (batch x ndf x sourceL)(batch x sourceL x queryL)
    # --> batch x ndf x queryL
    weightedContext = torch.bmm(context, attnT)

    return weightedContext, attn.view(batch_size, -1, ih, iw)


def train(dataloader, cnn_model, bert_encoder, batch_size, labels, optimizer, epoch, ixtoword, image_dir):
    train_function_start_time = time.time()
    cnn_model.train() #Sets the module in training mode.
    #rnn_model.train() #Sets the module in training mode.
    bert_encoder.train()
    
    s_total_loss0 = 0
    s_total_loss1 = 0
    w_total_loss0 = 0
    w_total_loss1 = 0
    
    # print("keyword |||||||||||||||||||||||||||||||")
    # print("len(dataloader) : " , len(dataloader) )
    # print(" count = " ,  (epoch + 1) * len(dataloader)  )
    # print("keyword |||||||||||||||||||||||||||||||")
    count = (epoch + 1) * len(dataloader)
    start_time = time.time()


    for step, data in enumerate(dataloader, 0):
    
    
        # Loading the first batch (number of batches/steps in an epoch is 183)
        bert_encoder.zero_grad()
        cnn_model.zero_grad()

        imgs, captions, cap_lens, class_ids, keys, b_input_ids, b_segments_ids = prepare_data(data)


        # words_features: batch_size x 256 x 17 x 17 ==> # image region features
        # sent_code: batch_size x 256                ==> # global image features
        words_features, sent_code = cnn_model(imgs[-1])
        # --> batch_size x nef x 17*17
        nef, att_sze = words_features.size(1), words_features.size(2)# 256, 17(16th of the whole image)
        # words_features = words_features.view(batch_size, nef, -1)

        #hidden = rnn_model.init_hidden(batch_size) # A tuple of 2 zero tensor of torch.Size([2, 48, 128])
        # words_emb: batch_size x nef x seq_len
        # sent_emb: batch_size x nef
        #words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

        #-------------------------------------------------------------------------------
                #---------------------------------------------------------
                        #-----------------------------------
        words_emb, sent_emb =bert_encoder(b_input_ids, b_segments_ids)
                        #-----------------------------------
                #---------------------------------------------------------
        #-------------------------------------------------------------------------------

        w_loss0, w_loss1, attn_maps = words_loss(words_features, words_emb, labels, cap_lens, class_ids, batch_size)
        w_total_loss0 += w_loss0.data
        w_total_loss1 += w_loss1.data
        loss = w_loss0 + w_loss1

        s_loss0, s_loss1 = sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        loss += s_loss0 + s_loss1
        s_total_loss0 += s_loss0.data
        s_total_loss1 += s_loss1.data
        #
        loss.backward()
        #
        # `clip_grad_norm` helps prevent
        # the exploding gradient problem in RNNs / LSTMs.
        #torch.nn.utils.clip_grad_norm(bert_encoder.parameters(), cfg.TRAIN.RNN_GRAD_CLIP)
        optimizer.step()

        if step % UPDATE_INTERVAL == 0:
            count = epoch * len(dataloader) + step

            # print ("====================================================")
            # print ("s_total_loss0 : " , s_total_loss0)
            # print ("s_total_loss0.item() : " , s_total_loss0.item())
            # print ("UPDATE_INTERVAL : " , UPDATE_INTERVAL)
            print ("s_total_loss0.item()/UPDATE_INTERVAL : " , s_total_loss0.item()/UPDATE_INTERVAL)
            print ("s_total_loss1.item()/UPDATE_INTERVAL : " , s_total_loss1.item()/UPDATE_INTERVAL)
            print ("w_total_loss0.item()/UPDATE_INTERVAL : " , w_total_loss0.item()/UPDATE_INTERVAL)
            print ("w_total_loss1.item()/UPDATE_INTERVAL : " , w_total_loss1.item()/UPDATE_INTERVAL)
            # print ("s_total_loss0/UPDATE_INTERVAL : " , s_total_loss0/UPDATE_INTERVAL)
            # print ("=====================================================")
            s_cur_loss0 = s_total_loss0.item() / UPDATE_INTERVAL
            s_cur_loss1 = s_total_loss1.item() / UPDATE_INTERVAL

            w_cur_loss0 = w_total_loss0.item() / UPDATE_INTERVAL
            w_cur_loss1 = w_total_loss1.item() / UPDATE_INTERVAL

            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
                    's_loss {:5.2f} {:5.2f} | '
                    'w_loss {:5.2f} {:5.2f}'
                    .format(epoch, step, len(dataloader),
                          elapsed * 1000. / UPDATE_INTERVAL,
                            s_cur_loss0, s_cur_loss1,
                            w_cur_loss0, w_cur_loss1))
            s_total_loss0 = 0
            s_total_loss1 = 0
            w_total_loss0 = 0
            w_total_loss1 = 0
            start_time = time.time()
            # attention Maps
            #Save image only every 8 epochs && Save it to The Drive
            if (epoch % 20 == 0):
                print("bulding images")
                img_set, _ = build_super_images(imgs[-1].cpu(), captions, ixtoword, attn_maps, att_sze)
                if img_set is not None:
                    im = Image.fromarray(img_set)
                    fullpath = '%s/attention_maps%d.png' % (image_dir, step)
                    im.save(fullpath)
                    mydriveimg = '/content/drive/My Drive/BertCUBImage'
                    drivepath = '%s/attention_maps%d.png' % (mydriveimg, epoch)
                    im.save(drivepath)
    print("keyTime |||||||||||||||||||||||||||||||")
    print("train_function_time : " , time.time() - train_function_start_time)
    print("KeyTime |||||||||||||||||||||||||||||||")
    return count


def evaluate(dataloader, cnn_model, bert_encoder, batch_size):
    cnn_model.eval()
    bert_encoder.eval()
    s_total_loss = 0
    w_total_loss = 0
    for step, data in enumerate(dataloader, 0):
        real_imgs, captions, cap_lens, class_ids, keys, b_input_ids, b_segments_ids = prepare_data(data)

        words_features, sent_code = cnn_model(real_imgs[-1])
        # nef = words_features.size(1)
        # words_features = words_features.view(batch_size, nef, -1)
        words_emb, sent_emb =bert_encoder(b_input_ids, b_segments_ids)



        w_loss0, w_loss1, attn = words_loss(words_features, words_emb, labels, cap_lens, class_ids, batch_size)
        w_total_loss += (w_loss0 + w_loss1).data

        s_loss0, s_loss1 = sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        s_total_loss += (s_loss0 + s_loss1).data

        if step == 50:
            break

    s_cur_loss = s_total_loss.item() / step
    w_cur_loss = w_total_loss.item() / step

    return s_cur_loss, w_cur_loss


def build_models():
    # build model ############################################################
    #text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    '''
    RNN_ENCODER(
    (encoder): Embedding(5450, 300)
    (drop): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True))
    '''
    print('build_models')
    image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    bert_encoder = BERT_ENCODER()

    labels = Variable(torch.LongTensor(range(batch_size)))
    '''
    A tensor of [0,1,2,3,...,47]
    '''
    start_epoch = 0
    if cfg.TRAIN.NET_E != '':
        state_dict = torch.load(cfg.TRAIN.NET_E)
        bert_encoder.load_state_dict(state_dict)
        print('Load ', cfg.TRAIN.NET_E)
        #
        name = cfg.TRAIN.NET_E.replace('text_encoder', 'image_encoder')
        state_dict = torch.load(name)
        image_encoder.load_state_dict(state_dict)
        print('Load ', name)

        istart = cfg.TRAIN.NET_E.rfind('_') + 8
        iend = cfg.TRAIN.NET_E.rfind('.')
        start_epoch = cfg.TRAIN.NET_E[istart:iend]
        start_epoch = int(start_epoch) + 1
        print('start_epoch', start_epoch)
    if cfg.CUDA:
        #text_encoder = text_encoder.cuda()
        image_encoder = image_encoder.cuda()
        labels = labels.cuda()
        bert_encoder = bert_encoder.cuda()

    return bert_encoder, image_encoder, labels, start_epoch


__name__ = "__main__"
if __name__ == "__main__":
    print('Using config:')
    pprint.pprint(cfg)

    UPDATE_INTERVAL = 200

    ##########################################################################
    now = datetime.datetime.now(dateutil.tz.tzlocal())
    timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
    output_dir = '../output/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

    model_dir = os.path.join(output_dir, 'Model')
    image_dir = os.path.join(output_dir, 'Image')
    mkdir_p(model_dir)
    mkdir_p(image_dir)

    torch.cuda.set_device(cfg.GPU_ID)
    cudnn.benchmark = True

    # Get data loader ##################################################
    imsize = 299
    batch_size = 48

    image_transform = transforms.Compose([transforms.Scale(355), transforms.RandomCrop(imsize), transforms.RandomHorizontalFlip()])
    
    dataset = TextDataset(cfg.DATA_DIR, 'train', base_size=cfg.TREE.BASE_SIZE, transform=image_transform, input_ids=input_ids, segments_ids=segments_ids, sentences=sentences )
    print(dataset.n_words, dataset.embeddings_num)
    assert dataset

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))
    #using prepare data functiont this dataloader yieldes:
    #imgs: a list of 1 tensor of size torch.Size([48, 3, 299, 299])
    #captons: a  tensor of size torch.Size([48, 18]), shorter filled with end words converted by word to index
    #cap_lens: a  tensor of size torch.Size([48]) , acual caps lens order from big to small (max is 18)
    #class_ids: a 48 ints list in range 0-200 of the classes
    #keys: a 48 string list  of the classes classes nammes crosspondening to the class_ids
    

    # # validation data #
    dataset_val = TextDataset(cfg.DATA_DIR, 'test', base_size=cfg.TREE.BASE_SIZE,transform=image_transform, input_ids=input_ids, segments_ids=segments_ids, sentences=sentences)
    print(dataset_val.n_words, dataset_val.embeddings_num)
    assert dataset_val
    dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, drop_last=True,shuffle=True, num_workers=int(cfg.WORKERS))

    # Train ##############################################################
    bert_encoder, image_encoder, labels, start_epoch = build_models()
    para = []
    
    for v in bert_encoder.parameters(): # 4 parameters
        if v.requires_grad:
            para.append(v)
    
    for v in image_encoder.parameters(): # 3 parameters
        if v.requires_grad:
            para.append(v)

    print ('requires_grad =' , len(para))
    # optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
    # At any point you can hit Ctrl + C to break out of training early.

    try:
        lr = cfg.TRAIN.ENCODER_LR #0.002
        print("keyword |||||||||||||||||||||||||||||||")
        print("Start_epoch : " , start_epoch)
        print("cfg.TRAIN.MAX_EPOCH : " , cfg.TRAIN.MAX_EPOCH )
        print("keyword |||||||||||||||||||||||||||||||")


        for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
            
            one_epoch_start_time = time.time()
            optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
            epoch_start_time = time.time()
            count = train(dataloader, image_encoder, bert_encoder, batch_size, labels, optimizer, epoch, dataset.ixtoword, image_dir)
            print('-' * 89)
            if len(dataloader_val) > 0:
                s_loss, w_loss = evaluate(dataloader_val, image_encoder, bert_encoder, batch_size)
                print('| end epoch {:3d} | valid loss ''{:5.2f} {:5.2f} | lr {:.5f}|'.format(epoch, s_loss, w_loss, lr))
            print('-' * 89)
            if lr > 0.0002 : #cfg.TRAIN.ENCODER_LR/10.:
                lr *= 0.98

            print("keyTime |||||||||||||||||||||||||||||||")
            print("one_epoch_time : " , time.time() - one_epoch_start_time)
            print("KeyTime |||||||||||||||||||||||||||||||")

            if (epoch % 20 == 0 or epoch == cfg.TRAIN.MAX_EPOCH or epoch == cfg.TRAIN.MAX_EPOCH-1 ):
                mydrivemodel = '/content/drive/My Drive/BertCUBModel'
                torch.save(image_encoder.state_dict(), '%s/image_encoder%d.pth' % (model_dir, epoch))
                torch.save(image_encoder.state_dict(), '%s/image_encoder%d.pth' % (mydrivemodel, epoch))
                torch.save(bert_encoder.state_dict(), '%s/text_encoder%d.pth' % (model_dir, epoch))
                torch.save(bert_encoder.state_dict(), '%s/text_encoder%d.pth' % (mydrivemodel, epoch))
                print('Save G/Ds models.')


    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')



In [ ]:
count = train(dataloader, image_encoder, bert_encoder, batch_size, labels, optimizer, epoch, dataset.ixtoword, image_dir)

In [ ]:
data = next(iter(dataloader))

In [ ]:
imgs, captions, cap_lens, class_ids, keys, b_input_ids, b_segments_ids = prepare_data(data)

In [ ]:
Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': '../data/birds',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'TEXT': {'CAPTIONS_PER_IMAGE': 10, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 18},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 50},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}
self.imsize [299]
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
self.imsize [299]
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
build_models
CNN_ENCODER
loaded_inception
Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Bert_ENCODER
Load pretrained model from  BertModel
requires_grad = 7
keyword |||||||||||||||||||||||||||||||
Start_epoch :  0
cfg.TRAIN.MAX_EPOCH :  600
keyword |||||||||||||||||||||||||||||||
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:978: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:987: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01944581151008606
s_total_loss1.item()/UPDATE_INTERVAL :  0.019677886962890623
w_total_loss0.item()/UPDATE_INTERVAL :  0.02603909969329834
w_total_loss1.item()/UPDATE_INTERVAL :  0.020277252197265627
| epoch   0 |     0/  184 batches | ms/batch  4.58 | s_loss  0.02  0.02 | w_loss  0.03  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  220.42531657218933
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  325.64459466934204
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   0 | valid loss  7.86  7.87 | lr 0.00200|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  350.91619515419006
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019367400407791138
s_total_loss1.item()/UPDATE_INTERVAL :  0.01936845302581787
w_total_loss0.item()/UPDATE_INTERVAL :  0.019382110834121703
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193767249584198
| epoch   1 |     0/  184 batches | ms/batch  4.82 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.82888722419739
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   1 | valid loss  7.86  7.86 | lr 0.00196|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.82555174827576
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019403358697891237
s_total_loss1.item()/UPDATE_INTERVAL :  0.019403971433639526
w_total_loss0.item()/UPDATE_INTERVAL :  0.01935318946838379
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935112476348877
| epoch   2 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.25210046768188
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   2 | valid loss  7.86  7.86 | lr 0.00192|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.09371447563171
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01935675024986267
s_total_loss1.item()/UPDATE_INTERVAL :  0.019356859922409056
w_total_loss0.item()/UPDATE_INTERVAL :  0.019341155290603637
w_total_loss1.item()/UPDATE_INTERVAL :  0.019338945150375365
| epoch   3 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.54735589027405
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   3 | valid loss  7.86  7.86 | lr 0.00188|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.41117215156555
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324768781661988
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325428009033203
w_total_loss0.item()/UPDATE_INTERVAL :  0.019402612447738648
w_total_loss1.item()/UPDATE_INTERVAL :  0.019402570724487304
| epoch   4 |     0/  184 batches | ms/batch  4.36 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.07228255271912
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   4 | valid loss  7.86  7.86 | lr 0.00184|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.0029125213623
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019336466789245606
s_total_loss1.item()/UPDATE_INTERVAL :  0.019336637258529663
w_total_loss0.item()/UPDATE_INTERVAL :  0.019298070669174196
w_total_loss1.item()/UPDATE_INTERVAL :  0.019299947023391724
| epoch   5 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.5748450756073
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   5 | valid loss  7.86  7.86 | lr 0.00181|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.52693796157837
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01937514781951904
s_total_loss1.item()/UPDATE_INTERVAL :  0.01937545895576477
w_total_loss0.item()/UPDATE_INTERVAL :  0.019394763708114625
w_total_loss1.item()/UPDATE_INTERVAL :  0.01939484238624573
| epoch   6 |     0/  184 batches | ms/batch  4.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.63466691970825
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   6 | valid loss  7.86  7.86 | lr 0.00177|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.62965893745422
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327225685119628
s_total_loss1.item()/UPDATE_INTERVAL :  0.019327253103256226
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934402346611023
w_total_loss1.item()/UPDATE_INTERVAL :  0.019343395233154297
| epoch   7 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.36361622810364
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   7 | valid loss  7.86  7.86 | lr 0.00174|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.23059368133545
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337979555130006
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933802604675293
w_total_loss0.item()/UPDATE_INTERVAL :  0.019414697885513307
w_total_loss1.item()/UPDATE_INTERVAL :  0.019413009881973267
| epoch   8 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.5503945350647
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   8 | valid loss  7.86  7.86 | lr 0.00170|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.58198928833008
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333211183547975
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933334469795227
w_total_loss0.item()/UPDATE_INTERVAL :  0.019364495277404786
w_total_loss1.item()/UPDATE_INTERVAL :  0.01936506271362305
| epoch   9 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.20327591896057
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   9 | valid loss  7.86  7.86 | lr 0.00167|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  113.95881724357605
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332839250564574
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332915544509888
w_total_loss0.item()/UPDATE_INTERVAL :  0.019337034225463866
w_total_loss1.item()/UPDATE_INTERVAL :  0.019337291717529296
| epoch  10 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.38318872451782
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  10 | valid loss  7.86  7.86 | lr 0.00163|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.28523015975952
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019335842132568358
s_total_loss1.item()/UPDATE_INTERVAL :  0.019335867166519166
w_total_loss0.item()/UPDATE_INTERVAL :  0.019346321821212768
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934728503227234
| epoch  11 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.67742681503296
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  11 | valid loss  7.86  7.86 | lr 0.00160|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.68802452087402
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329380989074708
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932960867881775
w_total_loss0.item()/UPDATE_INTERVAL :  0.01928188920021057
w_total_loss1.item()/UPDATE_INTERVAL :  0.019282506704330446
| epoch  12 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.14196348190308
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  12 | valid loss  7.86  7.86 | lr 0.00157|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.378413438797
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019310855865478517
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311089515686036
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318166971206665
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318394660949707
| epoch  13 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.9327974319458
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  13 | valid loss  7.86  7.86 | lr 0.00154|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.12397384643555
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334343671798707
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933441638946533
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319558143615724
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319642782211304
| epoch  14 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.97792363166809
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  14 | valid loss  7.86  7.86 | lr 0.00151|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.06809115409851
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019369282722473145
s_total_loss1.item()/UPDATE_INTERVAL :  0.019369406700134276
w_total_loss0.item()/UPDATE_INTERVAL :  0.019362126588821412
w_total_loss1.item()/UPDATE_INTERVAL :  0.019362248182296753
| epoch  15 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.89436888694763
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  15 | valid loss  7.86  7.86 | lr 0.00148|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.38795018196106
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931989550590515
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931984305381775
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931514382362366
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931515336036682
| epoch  16 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.88200497627258
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  16 | valid loss  7.86  7.86 | lr 0.00145|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.45942807197571
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932950019836426
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329513311386107
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329162836074828
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329053163528443
| epoch  17 |     0/  184 batches | ms/batch  4.40 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.53070163726807
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  17 | valid loss  7.86  7.86 | lr 0.00142|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.25876975059509
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334875345230103
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933493971824646
w_total_loss0.item()/UPDATE_INTERVAL :  0.019327468872070312
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932741641998291
| epoch  18 |     0/  184 batches | ms/batch  4.33 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.64077353477478
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  18 | valid loss  7.86  7.86 | lr 0.00139|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.84955430030823
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019315415620803834
s_total_loss1.item()/UPDATE_INTERVAL :  0.019315437078475953
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312971830368043
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931276321411133
| epoch  19 |     0/  184 batches | ms/batch  4.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.36970496177673
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  19 | valid loss  7.86  7.86 | lr 0.00136|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.265305519104
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0192984402179718
s_total_loss1.item()/UPDATE_INTERVAL :  0.019298543930053712
w_total_loss0.item()/UPDATE_INTERVAL :  0.019315757751464845
w_total_loss1.item()/UPDATE_INTERVAL :  0.019315818548202513
| epoch  20 |     0/  184 batches | ms/batch  4.35 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:288: RuntimeWarning: invalid value encountered in true_divide
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  228.87089562416077
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  327.7251043319702
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  20 | valid loss  7.86  7.86 | lr 0.00134|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  353.1852650642395
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931696891784668
s_total_loss1.item()/UPDATE_INTERVAL :  0.019316970109939574
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321626424789427
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321428537368776
| epoch  21 |     0/  184 batches | ms/batch  4.82 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.49985909461975
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  21 | valid loss  7.86  7.86 | lr 0.00131|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.00764441490173
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019307498931884767
s_total_loss1.item()/UPDATE_INTERVAL :  0.019307518005371095
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312095642089844
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931224822998047
| epoch  22 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.7789216041565
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  22 | valid loss  7.86  7.86 | lr 0.00128|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.75713849067688
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930907130241394
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309042692184447
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324420690536498
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932412266731262
| epoch  23 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.40695548057556
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  23 | valid loss  7.86  7.86 | lr 0.00126|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.11351776123047
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019344534873962402
s_total_loss1.item()/UPDATE_INTERVAL :  0.019344654083251953
w_total_loss0.item()/UPDATE_INTERVAL :  0.019340041875839233
w_total_loss1.item()/UPDATE_INTERVAL :  0.019340184926986696
| epoch  24 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.4506049156189
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  24 | valid loss  7.86  7.86 | lr 0.00123|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.09260082244873
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330978393554688
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331045150756836
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326590299606324
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326666593551634
| epoch  25 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.04186987876892
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  25 | valid loss  7.86  7.86 | lr 0.00121|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.06351804733276
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932404041290283
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932410717010498
w_total_loss0.item()/UPDATE_INTERVAL :  0.019340234994888305
w_total_loss1.item()/UPDATE_INTERVAL :  0.019340482950210573
| epoch  26 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.29820489883423
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  26 | valid loss  7.86  7.85 | lr 0.00118|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.12100553512573
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933042883872986
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330363273620605
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932016372680664
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193204402923584
| epoch  27 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.63258528709412
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  27 | valid loss  7.86  7.86 | lr 0.00116|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.6800799369812
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333935976028442
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334015846252443
w_total_loss0.item()/UPDATE_INTERVAL :  0.019331885576248167
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331822395324706
| epoch  28 |     0/  184 batches | ms/batch  4.42 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.7049822807312
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  28 | valid loss  7.86  7.86 | lr 0.00114|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.63709664344788
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334181547164916
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933423161506653
w_total_loss0.item()/UPDATE_INTERVAL :  0.019350970983505248
w_total_loss1.item()/UPDATE_INTERVAL :  0.019351269006729125
| epoch  29 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.51142239570618
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  29 | valid loss  7.86  7.86 | lr 0.00111|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.07112503051758
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019303590059280396
s_total_loss1.item()/UPDATE_INTERVAL :  0.019303690195083618
w_total_loss0.item()/UPDATE_INTERVAL :  0.019298152923583986
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929857611656189
| epoch  30 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.35507988929749
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  30 | valid loss  7.86  7.86 | lr 0.00109|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.29174280166626
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01934928297996521
s_total_loss1.item()/UPDATE_INTERVAL :  0.019349268674850463
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932822585105896
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327855110168456
| epoch  31 |     0/  184 batches | ms/batch  4.39 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.4527645111084
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  31 | valid loss  7.86  7.86 | lr 0.00107|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.64396166801453
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330472946166993
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330458641052248
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932583808898926
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932569980621338
| epoch  32 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.8074631690979
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  32 | valid loss  7.86  7.86 | lr 0.00105|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.97057962417603
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317679405212403
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931773781776428
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934548020362854
w_total_loss1.item()/UPDATE_INTERVAL :  0.019345070123672484
| epoch  33 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.80708050727844
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  33 | valid loss  7.86  7.86 | lr 0.00103|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.98481011390686
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324082136154174
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932409167289734
w_total_loss0.item()/UPDATE_INTERVAL :  0.019315296411514284
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931537628173828
| epoch  34 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.26109600067139
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  34 | valid loss  7.86  7.86 | lr 0.00101|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.85530018806458
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019348292350769042
s_total_loss1.item()/UPDATE_INTERVAL :  0.019348372220993042
w_total_loss0.item()/UPDATE_INTERVAL :  0.019353443384170534
w_total_loss1.item()/UPDATE_INTERVAL :  0.019353281259536743
| epoch  35 |     0/  184 batches | ms/batch  4.36 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.95972871780396
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  35 | valid loss  7.86  7.86 | lr 0.00099|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.05372095108032
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321603775024412
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321664571762084
w_total_loss0.item()/UPDATE_INTERVAL :  0.019308682680130005
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930843472480774
| epoch  36 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.34810876846313
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  36 | valid loss  7.86  7.85 | lr 0.00097|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.20249223709106
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341405630111694
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341410398483277
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931719422340393
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317063093185424
| epoch  37 |     0/  184 batches | ms/batch  4.48 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  96.40649557113647
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  37 | valid loss  7.86  7.86 | lr 0.00095|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.33039689064026
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932810425758362
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328122138977052
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933691143989563
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193367874622345
| epoch  38 |     0/  184 batches | ms/batch  4.38 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.11963605880737
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  38 | valid loss  7.86  7.86 | lr 0.00093|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.00628590583801
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333282709121703
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933332920074463
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934029459953308
w_total_loss1.item()/UPDATE_INTERVAL :  0.019340357780456542
| epoch  39 |     0/  184 batches | ms/batch  4.36 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.31938529014587
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  39 | valid loss  7.86  7.86 | lr 0.00091|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.22348856925964
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932411074638367
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324144124984743
w_total_loss0.item()/UPDATE_INTERVAL :  0.019336718320846557
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933668613433838
| epoch  40 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  225.58829998970032
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  324.06910586357117
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  40 | valid loss  7.86  7.86 | lr 0.00089|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  349.70646142959595
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320741891860962
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320842027664185
w_total_loss0.item()/UPDATE_INTERVAL :  0.019304856061935424
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930458664894104
| epoch  41 |     0/  184 batches | ms/batch  4.84 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.86981391906738
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  41 | valid loss  7.86  7.86 | lr 0.00087|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.5575909614563
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931393027305603
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931395649909973
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931616187095642
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316067695617677
| epoch  42 |     0/  184 batches | ms/batch  4.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.31013417243958
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  42 | valid loss  7.85  7.85 | lr 0.00086|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.09919476509094
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01934099555015564
s_total_loss1.item()/UPDATE_INTERVAL :  0.019340971708297728
w_total_loss0.item()/UPDATE_INTERVAL :  0.019344091415405273
w_total_loss1.item()/UPDATE_INTERVAL :  0.019344139099121093
| epoch  43 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.41556715965271
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  43 | valid loss  7.86  7.86 | lr 0.00084|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.49956750869751
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019301220178604125
s_total_loss1.item()/UPDATE_INTERVAL :  0.019301193952560424
w_total_loss0.item()/UPDATE_INTERVAL :  0.019303573369979857
w_total_loss1.item()/UPDATE_INTERVAL :  0.019303312301635744
| epoch  44 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.25117135047913
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  44 | valid loss  7.86  7.86 | lr 0.00082|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.76321768760681
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311453104019164
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311434030532836
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932402729988098
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323968887329103
| epoch  45 |     0/  184 batches | ms/batch  4.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.54570460319519
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  45 | valid loss  7.86  7.86 | lr 0.00081|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.64739894866943
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193380868434906
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338091611862184
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333478212356568
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333313703536987
| epoch  46 |     0/  184 batches | ms/batch  4.59 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.11429142951965
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  46 | valid loss  7.86  7.86 | lr 0.00079|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.43491053581238
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930259704589844
s_total_loss1.item()/UPDATE_INTERVAL :  0.019302610158920288
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931092619895935
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310804605484008
| epoch  47 |     0/  184 batches | ms/batch  4.43 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.09619450569153
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  47 | valid loss  7.86  7.85 | lr 0.00077|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.34948182106018
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323619604110717
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323636293411255
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329919815063476
w_total_loss1.item()/UPDATE_INTERVAL :  0.019330161809921264
| epoch  48 |     0/  184 batches | ms/batch  4.51 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.30101251602173
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  48 | valid loss  7.86  7.86 | lr 0.00076|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.51595902442932
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932188868522644
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932185173034668
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323405027389527
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932341456413269
| epoch  49 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.37202286720276
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  49 | valid loss  7.86  7.86 | lr 0.00074|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.43418025970459
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019331823587417602
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331846237182617
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933211326599121
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933203101158142
| epoch  50 |     0/  184 batches | ms/batch  4.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.53738903999329
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  50 | valid loss  7.86  7.86 | lr 0.00073|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.24563217163086
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933606743812561
s_total_loss1.item()/UPDATE_INTERVAL :  0.019336103200912474
w_total_loss0.item()/UPDATE_INTERVAL :  0.01935137152671814
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935141086578369
| epoch  51 |     0/  184 batches | ms/batch  4.40 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.5520989894867
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  51 | valid loss  7.86  7.86 | lr 0.00071|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.71643090248108
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931136965751648
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311379194259643
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931744933128357
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317264556884765
| epoch  52 |     0/  184 batches | ms/batch  4.35 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.78350448608398
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  52 | valid loss  7.86  7.86 | lr 0.00070|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.4741735458374
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931128978729248
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931131362915039
w_total_loss0.item()/UPDATE_INTERVAL :  0.019309163093566895
w_total_loss1.item()/UPDATE_INTERVAL :  0.019309226274490356
| epoch  53 |     0/  184 batches | ms/batch  4.35 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.50464129447937
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  53 | valid loss  7.85  7.85 | lr 0.00069|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.57931208610535
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324218034744264
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324246644973755
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320000410079956
w_total_loss1.item()/UPDATE_INTERVAL :  0.019320093393325806
| epoch  54 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.8800311088562
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  54 | valid loss  7.85  7.85 | lr 0.00067|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.38298654556274
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334665536880492
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334707260131836
w_total_loss0.item()/UPDATE_INTERVAL :  0.019337695837020875
w_total_loss1.item()/UPDATE_INTERVAL :  0.019337862730026245
| epoch  55 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.84962368011475
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  55 | valid loss  7.86  7.86 | lr 0.00066|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.95863509178162
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341229200363158
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341259002685546
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932357430458069
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323501586914062
| epoch  56 |     0/  184 batches | ms/batch  4.42 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.79318332672119
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  56 | valid loss  7.86  7.86 | lr 0.00065|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.21982169151306
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320472478866577
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320498704910278
w_total_loss0.item()/UPDATE_INTERVAL :  0.019302765130996703
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193026340007782
| epoch  57 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  96.02338147163391
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  57 | valid loss  7.86  7.86 | lr 0.00063|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.23108911514282
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320888519287108
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932090401649475
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933781623840332
w_total_loss1.item()/UPDATE_INTERVAL :  0.019337812662124632
| epoch  58 |     0/  184 batches | ms/batch  4.49 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  96.1704511642456
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  58 | valid loss  7.86  7.86 | lr 0.00062|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.69237613677979
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321857690811156
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321842193603514
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933881402015686
w_total_loss1.item()/UPDATE_INTERVAL :  0.019338690042495728
| epoch  59 |     0/  184 batches | ms/batch  4.42 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.61032772064209
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  59 | valid loss  7.86  7.86 | lr 0.00061|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.25628328323364
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019345282316207885
s_total_loss1.item()/UPDATE_INTERVAL :  0.019345312118530272
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332185983657837
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332261085510255
| epoch  60 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  223.5226218700409
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  322.00656747817993
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  60 | valid loss  7.86  7.86 | lr 0.00060|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  347.11649918556213
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330978393554688
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331021308898924
w_total_loss0.item()/UPDATE_INTERVAL :  0.019336036443710326
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336113929748534
| epoch  61 |     0/  184 batches | ms/batch  4.69 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.17992663383484
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  61 | valid loss  7.86  7.86 | lr 0.00058|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.3432354927063
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019345970153808595
s_total_loss1.item()/UPDATE_INTERVAL :  0.019345961809158325
w_total_loss0.item()/UPDATE_INTERVAL :  0.019348397254943847
w_total_loss1.item()/UPDATE_INTERVAL :  0.019348340034484865
| epoch  62 |     0/  184 batches | ms/batch  4.50 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  96.85009717941284
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  62 | valid loss  7.86  7.86 | lr 0.00057|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.62980318069458
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933111310005188
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193311071395874
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325008392333986
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932504177093506
| epoch  63 |     0/  184 batches | ms/batch  4.49 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.54401922225952
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  63 | valid loss  7.86  7.86 | lr 0.00056|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.02320885658264
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933328151702881
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333324432373046
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932965397834778
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329742193222046
| epoch  64 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.38326025009155
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  64 | valid loss  7.86  7.86 | lr 0.00055|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.88584613800049
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193208372592926
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320887327194215
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328261613845824
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328116178512572
| epoch  65 |     0/  184 batches | ms/batch  4.10 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.00788116455078
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  65 | valid loss  7.86  7.86 | lr 0.00054|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.78210639953613
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327845573425293
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193278968334198
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321885108947754
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932192325592041
| epoch  66 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.45278978347778
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  66 | valid loss  7.86  7.86 | lr 0.00053|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.9231367111206
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930539608001709
s_total_loss1.item()/UPDATE_INTERVAL :  0.019305368661880495
w_total_loss0.item()/UPDATE_INTERVAL :  0.019296690225601196
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929602265357971
| epoch  67 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.62395143508911
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  67 | valid loss  7.86  7.86 | lr 0.00052|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.59174370765686
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019293373823165892
s_total_loss1.item()/UPDATE_INTERVAL :  0.019293328523635866
w_total_loss0.item()/UPDATE_INTERVAL :  0.01929089069366455
w_total_loss1.item()/UPDATE_INTERVAL :  0.019290904998779296
| epoch  68 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.64311528205872
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  68 | valid loss  7.85  7.86 | lr 0.00051|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.93176460266113
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931913733482361
s_total_loss1.item()/UPDATE_INTERVAL :  0.019319143295288086
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326003789901732
w_total_loss1.item()/UPDATE_INTERVAL :  0.019325777292251586
| epoch  69 |     0/  184 batches | ms/batch  4.33 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.85730648040771
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  69 | valid loss  7.86  7.86 | lr 0.00050|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.16441059112549
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930901050567627
s_total_loss1.item()/UPDATE_INTERVAL :  0.019308918714523317
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310221672058106
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310417175292968
| epoch  70 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.77893137931824
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  70 | valid loss  7.86  7.86 | lr 0.00049|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.01701831817627
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019343819618225098
s_total_loss1.item()/UPDATE_INTERVAL :  0.019343804121017456
w_total_loss0.item()/UPDATE_INTERVAL :  0.019347349405288695
w_total_loss1.item()/UPDATE_INTERVAL :  0.019347381591796876
| epoch  71 |     0/  184 batches | ms/batch  4.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.16408181190491
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  71 | valid loss  7.86  7.86 | lr 0.00048|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.6055679321289
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019355195760726928
s_total_loss1.item()/UPDATE_INTERVAL :  0.019355210065841674
w_total_loss0.item()/UPDATE_INTERVAL :  0.019365984201431274
w_total_loss1.item()/UPDATE_INTERVAL :  0.01936610698699951
| epoch  72 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.8490834236145
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  72 | valid loss  7.86  7.86 | lr 0.00047|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.17390036582947
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019316409826278687
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931644320487976
w_total_loss0.item()/UPDATE_INTERVAL :  0.019327272176742554
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932731509208679
| epoch  73 |     0/  184 batches | ms/batch  4.41 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.55268597602844
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  73 | valid loss  7.86  7.86 | lr 0.00046|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.63656973838806
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01928306221961975
s_total_loss1.item()/UPDATE_INTERVAL :  0.019283108711242676
w_total_loss0.item()/UPDATE_INTERVAL :  0.019294768571853638
w_total_loss1.item()/UPDATE_INTERVAL :  0.0192945659160614
| epoch  74 |     0/  184 batches | ms/batch  4.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.83586740493774
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  74 | valid loss  7.86  7.86 | lr 0.00045|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.86078357696533
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019344325065612792
s_total_loss1.item()/UPDATE_INTERVAL :  0.019344358444213866
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934345841407776
w_total_loss1.item()/UPDATE_INTERVAL :  0.019343609809875487
| epoch  75 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.00734543800354
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  75 | valid loss  7.86  7.86 | lr 0.00044|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.60856580734253
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019310816526412963
s_total_loss1.item()/UPDATE_INTERVAL :  0.019310859441757203
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330629110336305
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331005811691285
| epoch  76 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.85501098632812
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  76 | valid loss  7.86  7.86 | lr 0.00043|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.14638996124268
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333287477493286
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333277940750123
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324593544006348
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932440161705017
| epoch  77 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.16791796684265
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  77 | valid loss  7.86  7.86 | lr 0.00042|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.54796648025513
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324245452880858
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324246644973755
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321879148483278
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321883916854857
| epoch  78 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.49694418907166
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  78 | valid loss  7.86  7.86 | lr 0.00041|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.82915306091309
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320964813232422
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320974349975584
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324873685836793
w_total_loss1.item()/UPDATE_INTERVAL :  0.019324606657028197
| epoch  79 |     0/  184 batches | ms/batch  4.53 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.99178624153137
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  79 | valid loss  7.86  7.86 | lr 0.00041|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.6974287033081
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320193529129028
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193202006816864
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323391914367674
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932318687438965
| epoch  80 |     0/  184 batches | ms/batch  4.38 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  223.43382859230042
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  320.75576305389404
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  80 | valid loss  7.86  7.86 | lr 0.00040|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  345.7842540740967
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019331218004226686
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933122634887695
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333168268203735
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333380460739135
| epoch  81 |     0/  184 batches | ms/batch  4.65 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.351482629776
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  81 | valid loss  7.86  7.86 | lr 0.00039|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.10030579566956
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931920051574707
s_total_loss1.item()/UPDATE_INTERVAL :  0.019319206476211548
w_total_loss0.item()/UPDATE_INTERVAL :  0.019331520795822142
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331413507461547
| epoch  82 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.38907265663147
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  82 | valid loss  7.85  7.86 | lr 0.00038|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.1301908493042
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931844711303711
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931846737861633
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320621490478515
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932062268257141
| epoch  83 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.28613758087158
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  83 | valid loss  7.86  7.86 | lr 0.00037|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.86054348945618
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019340118169784547
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934016227722168
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934289336204529
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934282898902893
| epoch  84 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.35437250137329
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  84 | valid loss  7.85  7.85 | lr 0.00037|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.28578639030457
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931365966796875
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931376338005066
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932033061981201
w_total_loss1.item()/UPDATE_INTERVAL :  0.019320305585861206
| epoch  85 |     0/  184 batches | ms/batch  4.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.27131700515747
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  85 | valid loss  7.86  7.86 | lr 0.00036|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.75818157196045
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933247685432434
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933251738548279
w_total_loss0.item()/UPDATE_INTERVAL :  0.019341219663619996
w_total_loss1.item()/UPDATE_INTERVAL :  0.019341211318969726
| epoch  86 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.61253237724304
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  86 | valid loss  7.86  7.86 | lr 0.00035|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.98581647872925
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338644742965698
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338656663894654
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933047890663147
w_total_loss1.item()/UPDATE_INTERVAL :  0.019330525398254396
| epoch  87 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.4118344783783
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  87 | valid loss  7.86  7.86 | lr 0.00034|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.9898362159729
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312176704406738
s_total_loss1.item()/UPDATE_INTERVAL :  0.019312191009521484
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312105178833007
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312349557876588
| epoch  88 |     0/  184 batches | ms/batch  4.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.4698805809021
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  88 | valid loss  7.86  7.86 | lr 0.00034|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.88481783866882
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019328489303588867
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328509569168092
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193254292011261
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932552695274353
| epoch  89 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.58057284355164
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  89 | valid loss  7.86  7.86 | lr 0.00033|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.1886932849884
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193200159072876
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932000756263733
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323400259017944
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932332992553711
| epoch  90 |     0/  184 batches | ms/batch  4.33 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.74290490150452
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  90 | valid loss  7.86  7.86 | lr 0.00032|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.95224237442017
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329575300216676
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329572916030883
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193268883228302
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932689070701599
| epoch  91 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.77791571617126
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  91 | valid loss  7.86  7.86 | lr 0.00032|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.3723361492157
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930763840675354
s_total_loss1.item()/UPDATE_INTERVAL :  0.019307584762573243
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323543310165406
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323525428771974
| epoch  92 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.78370809555054
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  92 | valid loss  7.86  7.86 | lr 0.00031|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.3124098777771
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330183267593382
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933019757270813
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932050347328186
w_total_loss1.item()/UPDATE_INTERVAL :  0.019320545196533204
| epoch  93 |     0/  184 batches | ms/batch  4.45 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.92639398574829
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  93 | valid loss  7.86  7.86 | lr 0.00031|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.52344298362732
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320988655090333
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320982694625854
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933977484703064
w_total_loss1.item()/UPDATE_INTERVAL :  0.019339948892593384
| epoch  94 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.97502994537354
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  94 | valid loss  7.86  7.86 | lr 0.00030|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.72330451011658
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932773232460022
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193277108669281
w_total_loss0.item()/UPDATE_INTERVAL :  0.019338847398757936
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933884620666504
| epoch  95 |     0/  184 batches | ms/batch  4.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.24034190177917
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  95 | valid loss  7.86  7.86 | lr 0.00029|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.27187919616699
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932334542274475
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323301315307618
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322398900985718
w_total_loss1.item()/UPDATE_INTERVAL :  0.019322404861450194
| epoch  96 |     0/  184 batches | ms/batch  4.53 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.30066275596619
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  96 | valid loss  7.86  7.86 | lr 0.00029|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.3517324924469
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933080792427063
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330854415893554
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330452680587768
w_total_loss1.item()/UPDATE_INTERVAL :  0.019330334663391114
| epoch  97 |     0/  184 batches | ms/batch  4.44 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.72908854484558
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  97 | valid loss  7.86  7.86 | lr 0.00028|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.89619159698486
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933165192604065
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331687688827516
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934224009513855
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934230446815491
| epoch  98 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  96.33125805854797
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  98 | valid loss  7.86  7.86 | lr 0.00028|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.17870354652405
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323849678039552
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323830604553224
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933125138282776
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331293106079103
| epoch  99 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.60530638694763
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  99 | valid loss  7.86  7.86 | lr 0.00027|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  118.42546057701111
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019339312314987183
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933932423591614
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317436218261718
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317641258239746
| epoch 100 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  226.723473072052
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  323.80058431625366
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 100 | valid loss  7.85  7.85 | lr 0.00027|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  348.88914132118225
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338817596435548
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338817596435548
w_total_loss0.item()/UPDATE_INTERVAL :  0.019342275857925414
w_total_loss1.item()/UPDATE_INTERVAL :  0.019342173337936402
| epoch 101 |     0/  184 batches | ms/batch  4.71 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.55466532707214
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 101 | valid loss  7.86  7.86 | lr 0.00026|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  121.23458981513977
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325751066207885
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325757026672365
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318281412124633
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318240880966186
| epoch 102 |     0/  184 batches | ms/batch  4.42 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.67532682418823
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 102 | valid loss  7.86  7.86 | lr 0.00025|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.61613345146179
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019309669733047485
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309648275375367
w_total_loss0.item()/UPDATE_INTERVAL :  0.019314342737197877
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931423544883728
| epoch 103 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.84859538078308
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 103 | valid loss  7.86  7.86 | lr 0.00025|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.42565655708313
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333410263061523
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333447217941283
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931894302368164
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319025278091432
| epoch 104 |     0/  184 batches | ms/batch  4.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.93509674072266
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 104 | valid loss  7.86  7.86 | lr 0.00024|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.95671963691711
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329748153686523
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329739809036253
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932982087135315
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329723119735718
| epoch 105 |     0/  184 batches | ms/batch  4.46 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.98021078109741
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 105 | valid loss  7.86  7.86 | lr 0.00024|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.02667880058289
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019307216405868532
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930717587471008
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930032134056091
w_total_loss1.item()/UPDATE_INTERVAL :  0.019300104379653932
| epoch 106 |     0/  184 batches | ms/batch  4.53 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.12308096885681
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 106 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.99289441108704
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019322199821472166
s_total_loss1.item()/UPDATE_INTERVAL :  0.019322233200073244
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310380220413207
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310232400894165
| epoch 107 |     0/  184 batches | ms/batch  4.30 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.6301064491272
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 107 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.45755171775818
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01934443473815918
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934444546699524
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328421354293822
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328399896621704
| epoch 108 |     0/  184 batches | ms/batch  4.39 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.48369717597961
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 108 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.43174695968628
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329835176467896
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329862594604494
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933568835258484
w_total_loss1.item()/UPDATE_INTERVAL :  0.019335867166519166
| epoch 109 |     0/  184 batches | ms/batch  4.41 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.9701087474823
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 109 | valid loss  7.86  7.86 | lr 0.00022|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.07643699645996
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019343852996826172
s_total_loss1.item()/UPDATE_INTERVAL :  0.019343878030776977
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193233060836792
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323182106018067
| epoch 110 |     0/  184 batches | ms/batch  4.41 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.14927172660828
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 110 | valid loss  7.86  7.86 | lr 0.00022|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.30662560462952
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341036081314086
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934105396270752
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932168483734131
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321800470352174
| epoch 111 |     0/  184 batches | ms/batch  4.33 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.79658150672913
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 111 | valid loss  7.86  7.86 | lr 0.00021|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.92331767082214
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019314184188842773
s_total_loss1.item()/UPDATE_INTERVAL :  0.019314180612564086
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193219792842865
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321961402893065
| epoch 112 |     0/  184 batches | ms/batch  4.38 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  95.38997483253479
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 112 | valid loss  7.86  7.86 | lr 0.00021|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  120.29045295715332
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334049224853517
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334062337875366
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329904317855834
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932982325553894
| epoch 113 |     0/  184 batches | ms/batch  4.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.0200719833374
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 113 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.30621671676636
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193479859828949
s_total_loss1.item()/UPDATE_INTERVAL :  0.019347991943359375
w_total_loss0.item()/UPDATE_INTERVAL :  0.019340473413467407
w_total_loss1.item()/UPDATE_INTERVAL :  0.019340416193008424
| epoch 114 |     0/  184 batches | ms/batch  4.52 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  94.92011022567749
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 114 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  119.26745009422302
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019331930875778197
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331914186477662
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332141876220704
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332060813903807
| epoch 115 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.59180998802185
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 115 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.61286926269531
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019336800575256347
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193367862701416
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334791898727416
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334768056869508
| epoch 116 |     0/  184 batches | ms/batch  4.33 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.64448499679565
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 116 | valid loss  7.85  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.3182053565979
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341219663619996
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341257810592653
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333184957504274
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333314895629884
| epoch 117 |     0/  184 batches | ms/batch  4.41 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.3389790058136
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 117 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.83904552459717
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932806372642517
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328074455261232
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333115816116332
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333111047744753
| epoch 118 |     0/  184 batches | ms/batch  4.12 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.56662034988403
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 118 | valid loss  7.85  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.29243493080139
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932362198829651
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323619604110717
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328174591064454
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328192472457886
| epoch 119 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.46145391464233
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 119 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.55522012710571
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019336637258529663
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933664321899414
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324699640274046
w_total_loss1.item()/UPDATE_INTERVAL :  0.019324711561203002
| epoch 120 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  218.0809874534607
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  311.7815523147583
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 120 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  335.94911909103394
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329766035079955
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932975769042969
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321489334106445
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321537017822264
| epoch 121 |     0/  184 batches | ms/batch  4.40 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.29146671295166
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 121 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.19292736053467
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338669776916503
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338692426681518
w_total_loss0.item()/UPDATE_INTERVAL :  0.019345030784606934
w_total_loss1.item()/UPDATE_INTERVAL :  0.019345070123672484
| epoch 122 |     0/  184 batches | ms/batch  4.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.18070197105408
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 122 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.79372835159302
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932207226753235
s_total_loss1.item()/UPDATE_INTERVAL :  0.019322067499160767
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318195581436156
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318209886550905
| epoch 123 |     0/  184 batches | ms/batch  4.38 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.21945333480835
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 123 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.33928179740906
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193264102935791
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932636260986328
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934160351753235
w_total_loss1.item()/UPDATE_INTERVAL :  0.019341440200805665
| epoch 124 |     0/  184 batches | ms/batch  4.11 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.00684905052185
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 124 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.02394366264343
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019315948486328127
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931595802307129
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328923225402833
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328597784042358
| epoch 125 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.49341249465942
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 125 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.63549709320068
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933883309364319
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338852167129515
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333223104476927
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933315634727478
| epoch 126 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.8913061618805
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 126 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.89674139022827
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931665062904358
s_total_loss1.item()/UPDATE_INTERVAL :  0.019316641092300416
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320241212844848
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932023048400879
| epoch 127 |     0/  184 batches | ms/batch  4.05 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.34081649780273
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 127 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.3327066898346
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931389808654785
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193139111995697
w_total_loss0.item()/UPDATE_INTERVAL :  0.019313143491744997
w_total_loss1.item()/UPDATE_INTERVAL :  0.019313052892684937
| epoch 128 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.9809193611145
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 128 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.88733983039856
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019319125413894654
s_total_loss1.item()/UPDATE_INTERVAL :  0.019319109916687012
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931964874267578
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931958079338074
| epoch 129 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.54897475242615
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 129 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.5028190612793
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932952880859375
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329519271850587
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933675765991211
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933677673339844
| epoch 130 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.5485508441925
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 130 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.50721168518066
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329010248184203
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328995943069457
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932750701904297
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327510595321656
| epoch 131 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.79482936859131
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 131 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.37725496292114
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019349449872970582
s_total_loss1.item()/UPDATE_INTERVAL :  0.019349470138549804
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934850335121155
w_total_loss1.item()/UPDATE_INTERVAL :  0.019348446130752563
| epoch 132 |     0/  184 batches | ms/batch  4.56 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.61439085006714
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 132 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.83207297325134
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931479334831238
s_total_loss1.item()/UPDATE_INTERVAL :  0.019314842224121095
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322699308395384
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932269215583801
| epoch 133 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.5897421836853
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 133 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.60768055915833
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019301099777221678
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930109620094299
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931334853172302
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931339144706726
| epoch 134 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.79183030128479
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 134 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.79179525375366
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311099052429198
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931111454963684
w_total_loss0.item()/UPDATE_INTERVAL :  0.019307804107666016
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930758833885193
| epoch 135 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.0137128829956
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 135 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.99741196632385
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193219530582428
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932195782661438
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933191657066345
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331750869750978
| epoch 136 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.26908588409424
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 136 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.6664514541626
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332818984985352
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332813024520876
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329801797866822
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329811334609984
| epoch 137 |     0/  184 batches | ms/batch  4.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.9749960899353
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 137 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.19340467453003
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019328240156173706
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932827353477478
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932647228240967
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326413869857787
| epoch 138 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.38330006599426
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 138 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.54745221138
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019316219091415406
s_total_loss1.item()/UPDATE_INTERVAL :  0.019316219091415406
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321228265762328
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321179389953612
| epoch 139 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.66744613647461
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 139 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.05454349517822
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019318909645080568
s_total_loss1.item()/UPDATE_INTERVAL :  0.019318903684616088
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930421233177185
w_total_loss1.item()/UPDATE_INTERVAL :  0.019304162263870238
| epoch 140 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  211.82105922698975
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  306.87600588798523
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 140 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  331.26331305503845
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324392080307007
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324421882629395
w_total_loss0.item()/UPDATE_INTERVAL :  0.019306656122207642
w_total_loss1.item()/UPDATE_INTERVAL :  0.019306735992431642
| epoch 141 |     0/  184 batches | ms/batch  4.69 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.4246015548706
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 141 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.11188769340515
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019310628175735475
s_total_loss1.item()/UPDATE_INTERVAL :  0.019310635328292847
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930719017982483
w_total_loss1.item()/UPDATE_INTERVAL :  0.019307245016098023
| epoch 142 |     0/  184 batches | ms/batch  4.50 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  93.30742192268372
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 142 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.47982120513916
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019340029954910277
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934001922607422
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328131675720214
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932819724082947
| epoch 143 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.02240872383118
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 143 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.17008972167969
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019346985816955566
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934698224067688
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933982014656067
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933979630470276
| epoch 144 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.20701837539673
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 144 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.39385104179382
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311189651489258
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931118845939636
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931704521179199
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317032098770143
| epoch 145 |     0/  184 batches | ms/batch  4.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.360107421875
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 145 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.3074381351471
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933318257331848
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333184957504274
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933393120765686
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333993196487428
| epoch 146 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.23918843269348
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 146 | valid loss  7.86  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.25591373443604
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01928780436515808
s_total_loss1.item()/UPDATE_INTERVAL :  0.01928775668144226
w_total_loss0.item()/UPDATE_INTERVAL :  0.019294458627700805
w_total_loss1.item()/UPDATE_INTERVAL :  0.019294356107711793
| epoch 147 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.19732642173767
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 147 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.48598647117615
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317547082901
s_total_loss1.item()/UPDATE_INTERVAL :  0.019317599534988402
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193285596370697
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328460693359376
| epoch 148 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.81404638290405
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 148 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.81923031806946
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019315863847732543
s_total_loss1.item()/UPDATE_INTERVAL :  0.019315885305404665
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316717386245727
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931667447090149
| epoch 149 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.1161720752716
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 149 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.27318382263184
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325679540634154
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325705766677855
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317935705184936
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317965507507324
| epoch 150 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.7088851928711
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 150 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.64744186401367
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931745409965515
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931747794151306
w_total_loss0.item()/UPDATE_INTERVAL :  0.019307307004928588
w_total_loss1.item()/UPDATE_INTERVAL :  0.019307290315628053
| epoch 151 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.03247809410095
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 151 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.15555930137634
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317216873168945
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931719422340393
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319825172424317
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319839477539062
| epoch 152 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.05373477935791
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 152 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.95084190368652
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019301079511642456
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930109620094299
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321848154067994
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321845769882204
| epoch 153 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.0971167087555
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 153 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.15856051445007
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019333076477050782
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933306336402893
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325891733169554
w_total_loss1.item()/UPDATE_INTERVAL :  0.019325925111770628
| epoch 154 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.00031471252441
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 154 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.9657576084137
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323469400405885
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323490858078003
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932745099067688
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327443838119508
| epoch 155 |     0/  184 batches | ms/batch  4.10 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.89144349098206
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 155 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.85455632209778
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019328869581222534
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328866004943848
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323850870132445
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323869943618774
| epoch 156 |     0/  184 batches | ms/batch  4.35 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.54872822761536
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 156 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.65758347511292
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019306167364120483
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930613994598389
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312347173690795
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931216597557068
| epoch 157 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.78750085830688
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 157 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.75262713432312
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334559440612794
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933456540107727
w_total_loss0.item()/UPDATE_INTERVAL :  0.019336719512939454
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336681365966796
| epoch 158 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.6330554485321
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 158 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.66003799438477
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330888986587524
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933089256286621
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317036867141722
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317121505737306
| epoch 159 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.43389654159546
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 159 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.45588493347168
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933537483215332
s_total_loss1.item()/UPDATE_INTERVAL :  0.019335380792617797
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328724145889282
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328752756118773
| epoch 160 |     0/  184 batches | ms/batch  4.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  215.48928833007812
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  309.46824312210083
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 160 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  333.6928651332855
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320188760757445
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932021737098694
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316896200180053
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931686520576477
| epoch 161 |     0/  184 batches | ms/batch  4.47 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.68896079063416
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 161 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.8162796497345
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019313063621520996
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313082695007325
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312667846679687
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931273818016052
| epoch 162 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.89290022850037
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 162 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.70937180519104
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321423768997193
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321409463882448
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319614171981813
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319597482681274
| epoch 163 |     0/  184 batches | ms/batch  4.07 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.45495295524597
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 163 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.28757929801941
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323571920394897
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323540925979613
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330843687057495
w_total_loss1.item()/UPDATE_INTERVAL :  0.019330769777297974
| epoch 164 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.40767049789429
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 164 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.03120422363281
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931793212890625
s_total_loss1.item()/UPDATE_INTERVAL :  0.019317946434020995
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320354461669922
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932029128074646
| epoch 165 |     0/  184 batches | ms/batch  4.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.93613839149475
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 165 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.0993480682373
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325500726699828
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325484037399292
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324816465377807
w_total_loss1.item()/UPDATE_INTERVAL :  0.019324826002120973
| epoch 166 |     0/  184 batches | ms/batch  4.12 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.84316158294678
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 166 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.158367395401
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019326046705245972
s_total_loss1.item()/UPDATE_INTERVAL :  0.019326053857803345
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316034317016603
w_total_loss1.item()/UPDATE_INTERVAL :  0.019315996170043946
| epoch 167 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.1641776561737
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 167 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.48659420013428
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933563709259033
s_total_loss1.item()/UPDATE_INTERVAL :  0.019335660934448242
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332095384597778
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332162141799926
| epoch 168 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.79656982421875
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 168 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.2891788482666
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019313076734542845
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313071966171266
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931163191795349
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931158661842346
| epoch 169 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.21985697746277
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 169 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.44109988212585
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320868253707886
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320887327194215
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316902160644533
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316781759262085
| epoch 170 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.0772123336792
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 170 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.36596202850342
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931246280670166
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931249737739563
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310593605041504
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310547113418577
| epoch 171 |     0/  184 batches | ms/batch  4.11 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.29910659790039
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 171 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.42049312591553
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019339295625686644
s_total_loss1.item()/UPDATE_INTERVAL :  0.019339302778244017
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933330178260803
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333339929580688
| epoch 172 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.31687211990356
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 172 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.65415239334106
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930137276649475
s_total_loss1.item()/UPDATE_INTERVAL :  0.019301306009292602
w_total_loss0.item()/UPDATE_INTERVAL :  0.019304641485214234
w_total_loss1.item()/UPDATE_INTERVAL :  0.019304572343826293
| epoch 173 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.37863993644714
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 173 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.41510438919067
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019276586771011354
s_total_loss1.item()/UPDATE_INTERVAL :  0.019276599884033203
w_total_loss0.item()/UPDATE_INTERVAL :  0.0192752206325531
w_total_loss1.item()/UPDATE_INTERVAL :  0.019275423288345337
| epoch 174 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.1192717552185
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 174 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.1464421749115
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933608055114746
s_total_loss1.item()/UPDATE_INTERVAL :  0.019336100816726685
w_total_loss0.item()/UPDATE_INTERVAL :  0.019335752725601195
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933576226234436
| epoch 175 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.44756054878235
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 175 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.71400427818298
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019309476613998414
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309521913528443
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310102462768555
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310169219970703
| epoch 176 |     0/  184 batches | ms/batch  4.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.37440133094788
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 176 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.82341766357422
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019310346841812133
s_total_loss1.item()/UPDATE_INTERVAL :  0.019310342073440553
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932112216949463
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932109713554382
| epoch 177 |     0/  184 batches | ms/batch  4.39 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.54601740837097
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 177 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.11512041091919
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933282732963562
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332852363586426
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317344427108765
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317327737808226
| epoch 178 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.06797313690186
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 178 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.29454278945923
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312852621078493
s_total_loss1.item()/UPDATE_INTERVAL :  0.019312881231307984
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930325388908386
w_total_loss1.item()/UPDATE_INTERVAL :  0.019303385019302368
| epoch 179 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.52830982208252
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 179 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.9353415966034
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323840141296386
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932386517524719
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318896532058715
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193189001083374
| epoch 180 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.73372411727905
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  311.9784595966339
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 180 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  336.5493462085724
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930979371070862
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309730529785157
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317593574523926
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931759834289551
| epoch 181 |     0/  184 batches | ms/batch  4.68 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.05387282371521
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 181 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.06938743591309
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327359199523927
s_total_loss1.item()/UPDATE_INTERVAL :  0.019327378273010253
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318174123764038
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318112134933473
| epoch 182 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.93968081474304
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 182 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.07615876197815
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932227611541748
s_total_loss1.item()/UPDATE_INTERVAL :  0.019322270154953004
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932858467102051
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328560829162598
| epoch 183 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.9549388885498
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 183 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.90118622779846
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325447082519532
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325486421585082
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932362198829651
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323526620864867
| epoch 184 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.29578614234924
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 184 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.57093381881714
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019319270849227906
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931924343109131
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932766079902649
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932756781578064
| epoch 185 |     0/  184 batches | ms/batch  4.40 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.45150804519653
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 185 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.52662348747253
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325517416000366
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932552456855774
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932797074317932
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327954053878785
| epoch 186 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.95971059799194
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 186 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.95633888244629
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930872440338135
s_total_loss1.item()/UPDATE_INTERVAL :  0.019308727979660035
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930487036705017
w_total_loss1.item()/UPDATE_INTERVAL :  0.019304776191711427
| epoch 187 |     0/  184 batches | ms/batch  4.14 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.28744864463806
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 187 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.39055371284485
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321926832199098
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321926832199098
w_total_loss0.item()/UPDATE_INTERVAL :  0.019327124357223512
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932706356048584
| epoch 188 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.93172001838684
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 188 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.04380702972412
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332998991012575
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333022832870483
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193331515789032
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333091974258424
| epoch 189 |     0/  184 batches | ms/batch  4.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.27718496322632
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 189 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.47409057617188
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193038547039032
s_total_loss1.item()/UPDATE_INTERVAL :  0.019303866624832154
w_total_loss0.item()/UPDATE_INTERVAL :  0.019303239583969116
w_total_loss1.item()/UPDATE_INTERVAL :  0.019303036928176878
| epoch 190 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.30605173110962
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 190 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.34382677078247
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341922998428344
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341936111450197
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326190948486328
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326130151748656
| epoch 191 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.34500169754028
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 191 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.54489612579346
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932066082954407
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320694208145143
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321709871292114
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321694374084472
| epoch 192 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.69156551361084
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 192 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.81052923202515
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01929897665977478
s_total_loss1.item()/UPDATE_INTERVAL :  0.019298948049545288
w_total_loss0.item()/UPDATE_INTERVAL :  0.019301085472106932
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930108666419983
| epoch 193 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.90953350067139
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 193 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.07342529296875
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932127833366394
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932126998901367
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319831132888793
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319981336593628
| epoch 194 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.66803240776062
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 194 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.87234425544739
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320603609085083
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932058811187744
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323405027389527
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323391914367674
| epoch 195 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.84850144386292
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 195 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.07008838653564
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311610460281372
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931161284446716
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930726408958435
w_total_loss1.item()/UPDATE_INTERVAL :  0.019307161569595336
| epoch 196 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.36219000816345
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 196 | valid loss  7.86  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.60706663131714
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329055547714233
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329060316085816
w_total_loss0.item()/UPDATE_INTERVAL :  0.019336310625076295
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933627724647522
| epoch 197 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.37678384780884
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 197 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.68781304359436
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931340217590332
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313353300094604
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316745996475218
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316626787185667
| epoch 198 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.46809387207031
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 198 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.63514232635498
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337137937545778
s_total_loss1.item()/UPDATE_INTERVAL :  0.019337142705917357
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933869957923889
w_total_loss1.item()/UPDATE_INTERVAL :  0.019338693618774414
| epoch 199 |     0/  184 batches | ms/batch  4.36 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.79642653465271
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 199 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.84054160118103
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321664571762084
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321703910827638
w_total_loss0.item()/UPDATE_INTERVAL :  0.019314891099929808
w_total_loss1.item()/UPDATE_INTERVAL :  0.019314908981323244
| epoch 200 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.89982795715332
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  311.49752283096313
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 200 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  335.97231006622314
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338840246200563
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338887929916383
w_total_loss0.item()/UPDATE_INTERVAL :  0.019327596426010132
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327481985092165
| epoch 201 |     0/  184 batches | ms/batch  4.77 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.74900126457214
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 201 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.15353345870972
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019322983026504516
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932299494743347
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932724952697754
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932719588279724
| epoch 202 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.37889671325684
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 202 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.661612033844
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932514548301697
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325162172317504
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931883454322815
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318851232528685
| epoch 203 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.914724111557
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 203 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.51459097862244
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193028199672699
s_total_loss1.item()/UPDATE_INTERVAL :  0.019302830696105958
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931040167808533
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931041359901428
| epoch 204 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.40818428993225
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 204 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.01649951934814
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324440956115723
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932441592216492
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326677322387697
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932671070098877
| epoch 205 |     0/  184 batches | ms/batch  4.66 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.23944330215454
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 205 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.53007292747498
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931301712989807
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313035011291505
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932652950286865
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326499700546264
| epoch 206 |     0/  184 batches | ms/batch  4.28 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.68048763275146
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 206 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.04981803894043
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311782121658325
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311771392822266
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931735038757324
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931756138801575
| epoch 207 |     0/  184 batches | ms/batch  4.32 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.46614456176758
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 207 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.81338620185852
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932034969329834
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932034134864807
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321494102478028
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321556091308593
| epoch 208 |     0/  184 batches | ms/batch  4.29 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.18720650672913
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 208 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.51895999908447
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933374524116516
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333759546279906
w_total_loss0.item()/UPDATE_INTERVAL :  0.019343104362487793
w_total_loss1.item()/UPDATE_INTERVAL :  0.019343096017837524
| epoch 209 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.4899172782898
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 209 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.84013366699219
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933217644691467
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933220386505127
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330343008041383
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933024764060974
| epoch 210 |     0/  184 batches | ms/batch  4.38 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.93595862388611
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 210 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.54080724716187
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932123064994812
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932120442390442
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931961178779602
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319615364074706
| epoch 211 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.32783961296082
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 211 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.83171057701111
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932278037071228
s_total_loss1.item()/UPDATE_INTERVAL :  0.019322772026062012
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932596802711487
w_total_loss1.item()/UPDATE_INTERVAL :  0.019325942993164064
| epoch 212 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.08445811271667
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 212 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.43530082702637
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337247610092163
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933726906776428
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332395792007448
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933237671852112
| epoch 213 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.1058304309845
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 213 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.34019017219543
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327694177627565
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932770848274231
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323095083236694
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323123693466185
| epoch 214 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.89045834541321
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 214 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.15739798545837
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019336992502212526
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193369996547699
w_total_loss0.item()/UPDATE_INTERVAL :  0.019336246252059937
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336262941360472
| epoch 215 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.97589159011841
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 215 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.16657614707947
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933887004852295
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338871240615844
w_total_loss0.item()/UPDATE_INTERVAL :  0.019337345361709595
w_total_loss1.item()/UPDATE_INTERVAL :  0.019337190389633177
| epoch 216 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.11626720428467
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 216 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.3914566040039
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019345645904541017
s_total_loss1.item()/UPDATE_INTERVAL :  0.019345654249191283
w_total_loss0.item()/UPDATE_INTERVAL :  0.019357681274414062
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935775876045227
| epoch 217 |     0/  184 batches | ms/batch  4.24 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.45926547050476
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 217 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  117.0061445236206
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332824945449828
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332810640335082
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325374364852904
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932541251182556
| epoch 218 |     0/  184 batches | ms/batch  4.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.77158427238464
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 218 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.99967813491821
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933911204338074
s_total_loss1.item()/UPDATE_INTERVAL :  0.019339102506637573
w_total_loss0.item()/UPDATE_INTERVAL :  0.019339312314987183
w_total_loss1.item()/UPDATE_INTERVAL :  0.019339290857315065
| epoch 219 |     0/  184 batches | ms/batch  4.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.23477673530579
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 219 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.93397784233093
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933457374572754
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193346107006073
w_total_loss0.item()/UPDATE_INTERVAL :  0.019340044260025023
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934008240699768
| epoch 220 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.272851228714
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  311.3080916404724
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 220 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  335.6349558830261
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019328365325927733
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328351020812987
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328500032424926
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328387975692748
| epoch 221 |     0/  184 batches | ms/batch  4.73 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.0341010093689
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 221 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.10778856277466
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932836890220642
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328402280807497
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323906898498534
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323900938034058
| epoch 222 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.31771659851074
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 222 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.42161417007446
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338916540145874
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933892846107483
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332584142684937
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332596063613892
| epoch 223 |     0/  184 batches | ms/batch  4.11 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.34950041770935
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 223 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.61339688301086
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932522773742676
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325231313705445
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312113523483276
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312162399291993
| epoch 224 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.1023199558258
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 224 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.25724673271179
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019322619438171387
s_total_loss1.item()/UPDATE_INTERVAL :  0.019322644472122192
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931912064552307
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319100379943846
| epoch 225 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.00852012634277
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 225 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.10776376724243
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330205917358397
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330220222473146
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329801797866822
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329813718795778
| epoch 226 |     0/  184 batches | ms/batch  4.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.330557346344
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 226 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.46405577659607
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931093096733093
s_total_loss1.item()/UPDATE_INTERVAL :  0.019310964345932005
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931027412414551
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310094118118286
| epoch 227 |     0/  184 batches | ms/batch  4.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.09593558311462
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 227 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.13320183753967
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933560013771057
s_total_loss1.item()/UPDATE_INTERVAL :  0.019335588216781618
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326714277267457
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326595067977907
| epoch 228 |     0/  184 batches | ms/batch  4.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.31308245658875
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 228 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.31327319145203
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933857798576355
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338587522506712
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330180883407592
w_total_loss1.item()/UPDATE_INTERVAL :  0.019330133199691773
| epoch 229 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.26014375686646
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 229 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.35758328437805
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931622385978699
s_total_loss1.item()/UPDATE_INTERVAL :  0.019316166639328003
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317502975463866
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931759238243103
| epoch 230 |     0/  184 batches | ms/batch  4.20 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.26853561401367
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 230 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.34663343429565
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332066774368287
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332088232040405
w_total_loss0.item()/UPDATE_INTERVAL :  0.019331547021865843
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331501722335817
| epoch 231 |     0/  184 batches | ms/batch  4.08 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.8883740901947
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 231 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.3065915107727
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019314570426940916
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931459069252014
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932301163673401
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932284712791443
| epoch 232 |     0/  184 batches | ms/batch  4.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.37433576583862
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 232 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.63951754570007
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338903427124025
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193389368057251
w_total_loss0.item()/UPDATE_INTERVAL :  0.019335566759109496
w_total_loss1.item()/UPDATE_INTERVAL :  0.019335612058639526
| epoch 233 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.97356152534485
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 233 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.24518847465515
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019295735359191893
s_total_loss1.item()/UPDATE_INTERVAL :  0.019295705556869505
w_total_loss0.item()/UPDATE_INTERVAL :  0.019305464029312134
w_total_loss1.item()/UPDATE_INTERVAL :  0.019305448532104492
| epoch 234 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.56219363212585
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 234 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.67566251754761
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019299873113632203
s_total_loss1.item()/UPDATE_INTERVAL :  0.019299861192703247
w_total_loss0.item()/UPDATE_INTERVAL :  0.019292279481887817
w_total_loss1.item()/UPDATE_INTERVAL :  0.019292298555374145
| epoch 235 |     0/  184 batches | ms/batch  4.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.52764439582825
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 235 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.733389377594
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019308756589889526
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930872440338135
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931618332862854
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316239356994627
| epoch 236 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  92.28292417526245
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 236 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  116.51669335365295
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317426681518556
s_total_loss1.item()/UPDATE_INTERVAL :  0.019317439794540404
w_total_loss0.item()/UPDATE_INTERVAL :  0.019326186180114745
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326144456863405
| epoch 237 |     0/  184 batches | ms/batch  4.22 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.63032722473145
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 237 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.77386355400085
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930364966392517
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930362343788147
w_total_loss0.item()/UPDATE_INTERVAL :  0.019307740926742555
w_total_loss1.item()/UPDATE_INTERVAL :  0.019307838678359987
| epoch 238 |     0/  184 batches | ms/batch  4.23 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.36099672317505
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 238 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.54079484939575
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337061643600464
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933707356452942
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933558464050293
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933556914329529
| epoch 239 |     0/  184 batches | ms/batch  4.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  91.44328165054321
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 239 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  115.67916798591614
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931886315345764
s_total_loss1.item()/UPDATE_INTERVAL :  0.019318858385086058
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933510422706604
w_total_loss1.item()/UPDATE_INTERVAL :  0.019335174560546876
| epoch 240 |     0/  184 batches | ms/batch  4.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  215.43769121170044
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  309.36272740364075
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 240 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  333.4256155490875
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019340839385986328
s_total_loss1.item()/UPDATE_INTERVAL :  0.019340845346450804
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932774543762207
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327679872512816
| epoch 241 |     0/  184 batches | ms/batch  4.60 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.9034035205841
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 241 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.74507522583008
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332112073898317
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332109689712523
w_total_loss0.item()/UPDATE_INTERVAL :  0.019338579177856446
w_total_loss1.item()/UPDATE_INTERVAL :  0.019338507652282715
| epoch 242 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.5163516998291
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 242 | valid loss  7.85  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.535076379776
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338711500167846
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338706731796263
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933938980102539
w_total_loss1.item()/UPDATE_INTERVAL :  0.019339427947998047
| epoch 243 |     0/  184 batches | ms/batch  4.08 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.39703249931335
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 243 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.30765867233276
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334473609924317
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334487915039063
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334187507629396
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334187507629396
| epoch 244 |     0/  184 batches | ms/batch  4.11 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.3491325378418
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 244 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.16480851173401
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931696891784668
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931693434715271
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316869974136352
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316874742507935
| epoch 245 |     0/  184 batches | ms/batch  4.08 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.67261719703674
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 245 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.61237978935242
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019293432235717774
s_total_loss1.item()/UPDATE_INTERVAL :  0.019293404817581176
w_total_loss0.item()/UPDATE_INTERVAL :  0.019286479949951172
w_total_loss1.item()/UPDATE_INTERVAL :  0.019286551475524903
| epoch 246 |     0/  184 batches | ms/batch  4.21 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.5631730556488
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 246 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.50670075416565
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019300352334976196
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930036664009094
w_total_loss0.item()/UPDATE_INTERVAL :  0.019305295944213867
w_total_loss1.item()/UPDATE_INTERVAL :  0.019305248260498047
| epoch 247 |     0/  184 batches | ms/batch  4.16 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.35510087013245
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 247 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.24228119850159
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932876229286194
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328793287277223
w_total_loss0.item()/UPDATE_INTERVAL :  0.019315115213394164
w_total_loss1.item()/UPDATE_INTERVAL :  0.019315077066421507
| epoch 248 |     0/  184 batches | ms/batch  4.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  90.72857093811035
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 248 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  114.60247206687927
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019338196516036986
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933820724487305
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934013843536377
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934015154838562
| epoch 249 |     0/  184 batches | ms/batch  4.07 | s_loss  0.02  0.02 | w_loss  0.02  0.02